# CNV MMRF CoMMpass — Pipeline (V7, reproducible)

This notebook is a **scientifically rigorous and reproducible** CNV pipeline for the **MMRF CoMMpass** project (via GDC).

## What V7 improves (key points)
- **CNV ↔ Patient join**: builds `file_metadata.tsv` with `cases.submitter_id` (Participant_ID) directly from the GDC API (no UUID heuristics).
- **Single source of truth**: produces a canonical `df_cnvs_final` with schema/type checks and chromosome normalization.
- **No synthetic fallbacks**: no example data is injected.
- **Biologically meaningful recurrence**: recurrence by **cytoband overlap** and by **fixed genomic bins** (instead of exact `chr:start-end`).
- **Survival robustness**: avoids converting missingness (NA) into 0; includes FDR control and PH checks (when applicable).
- **Predictive models (optional)**: anti-leakage survival risk prediction (penalized Cox) and ISS stage classification (macro-F1/accuracy).



In [ ]:

# =========================
# 0) SETUP (Colab-friendly)
# =========================
import sys, subprocess, importlib.util, os, json, hashlib

def _ensure(pkg: str, pip_name: str | None = None):
    if importlib.util.find_spec(pkg) is None:
        subprocess.check_call([sys.executable, "-m", "pip", "-q", "install", (pip_name or pkg)])

for pkg, pipn in [
    ("requests", None),
    ("tqdm", None),
    ("pandas", None),
    ("numpy", None),
    ("matplotlib", None),
    ("sklearn", "scikit-learn"),
    ("lifelines", None),
    ("pyarrow", None),
    ("pyranges", None),
]:
    _ensure(pkg, pipn)

import requests
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pyranges as pr

print("[OK] Dependências carregadas.")


In [ ]:

# =========================
# 1) CONFIG + PROVENANCE
# =========================
from datetime import datetime

RUN_ID = "20260123_154712"
BASE_OUT_DIR = os.path.join("outputs", f"run_{RUN_ID}")
RAW_DIR  = os.path.join(BASE_OUT_DIR, "raw")
PROC_DIR = os.path.join(BASE_OUT_DIR, "processed")
RES_DIR  = os.path.join(BASE_OUT_DIR, "results")
LOG_DIR  = os.path.join(BASE_OUT_DIR, "logs")

# aliases (compatibilidade com versões antigas)
OUT_DIR = BASE_OUT_DIR
RESULTS_DIR = RES_DIR
PROCESSED_DIR = PROC_DIR

for d in [BASE_OUT_DIR, RAW_DIR, PROC_DIR, RES_DIR, LOG_DIR]:
    os.makedirs(d, exist_ok=True)

GDC_FILES_URL = "https://api.gdc.cancer.gov/files"
GDC_DATA_URL  = "https://api.gdc.cancer.gov/data"

PROJECT_ID = "MMRF-COMMPASS"
DATA_CATEGORY = "Copy Number Variation"

DOWNLOAD_DIR = os.path.join(RAW_DIR, "mmrf_cnv_data")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

THRESHOLDS = {
    "LOSS_HOMDEL": -1.5,
    "LOSS_1COPY_MAX": -0.5,
    "NEUTRAL_MIN": -0.2,
    "NEUTRAL_MAX":  0.2,
    "GAIN_1COPY_MIN": 0.5,
    "GAIN_AMP": 1.0
}

BIN_SIZE = 1_000_000
TOP_N_REGIONS = 50

# parâmetros do módulo data-driven (recurrent CNV)
TOP_K_TOPCNV = 10       # quantas regiões recorrentes testar (bins)
TOP_KM_PLOTS  = 10      # quantos KM plots salvar (seleção por p/frequência)
MIN_OVERLAP_BP = 1      # presença em bin: overlap mínimo em bp

SEED = 42
np.random.seed(SEED)

params = {
    "RUN_ID": RUN_ID,
    "PROJECT_ID": PROJECT_ID,
    "DATA_CATEGORY": DATA_CATEGORY,
    "THRESHOLDS": THRESHOLDS,
    "BIN_SIZE": BIN_SIZE,
    "SEED": SEED
}
with open(os.path.join(LOG_DIR, "run_params.json"), "w") as f:
    json.dump(params, f, indent=2)

print(f"[OK] RUN_ID={RUN_ID}")
print(f"[OK] Saídas em: {BASE_OUT_DIR}")


In [ ]:

# =========================
# 2) UTILITIES (auditáveis)
# =========================
def export_df_stats(df: pd.DataFrame, label: str, out_dir: str = None, n_head: int = 5):
    # Pequeno relatório (shape, dtypes, NA, head) para auditoria
    out_dir = out_dir or LOG_DIR
    out = []
    out.append(f"label: {label}")
    out.append(f"shape: {df.shape}")
    out.append("columns:")
    out.append(", ".join(map(str, df.columns.tolist())))
    out.append("\nmissingness (top 20):")
    miss = df.isna().mean().sort_values(ascending=False).head(20)
    out.append(miss.to_string())
    out.append("\ndtypes:")
    out.append(df.dtypes.astype(str).to_string())
    out.append("\nhead:")
    out.append(df.head(n_head).to_string(index=False))

    path = os.path.join(out_dir, f"{label}__stats.txt")
    with open(path, "w") as f:
        f.write("\n".join(out))
    print(f"[OK] stats → {path}")

def md5_file(path: str, chunk_size: int = 1<<20) -> str:
    h = hashlib.md5()
    with open(path, "rb") as f:
        while True:
            b = f.read(chunk_size)
            if not b:
                break
            h.update(b)
    return h.hexdigest()

def safe_col(df: pd.DataFrame, candidates: list[str]):
    for c in candidates:
        if c in df.columns:
            return c
    return None

print("[OK] Utilitários prontos.")


In [ ]:

# ==========================================
# 3) GDC: LIST FILES + METADATA (JOIN)
# ==========================================
filters = {
    "op": "and",
    "content": [
        {"op": "=",  "content": {"field": "cases.project.project_id", "value": [PROJECT_ID]}},
        {"op": "=",  "content": {"field": "data_category",            "value": [DATA_CATEGORY]}},
        {"op": "in", "content": {"field": "access",                  "value": ["open"]}},
    ]
}

fields = [
    "file_id","file_name","data_category","data_type","data_format","file_size","md5sum",
    "cases.submitter_id","cases.case_id"
]
fields_str = ",".join(fields)

def gdc_list_files(filters: dict, fields: str, page_size: int = 2000) -> pd.DataFrame:
    payload = {
        "filters": filters,
        "fields": fields,
        "format": "JSON",
        "size": page_size
    }
    r = requests.post(GDC_FILES_URL, json=payload, timeout=120)
    r.raise_for_status()
    j = r.json()
    hits = j["data"]["hits"]
    rows = []
    for h in hits:
        case_submitter = None
        case_id = None
        if isinstance(h.get("cases"), list) and len(h["cases"]) > 0:
            case_submitter = h["cases"][0].get("submitter_id")
            case_id = h["cases"][0].get("case_id")
        rows.append({
            "file_id": h.get("file_id"),
            "file_name": h.get("file_name"),
            "data_category": h.get("data_category"),
            "data_type": h.get("data_type"),
            "data_format": h.get("data_format"),
            "file_size": h.get("file_size"),
            "md5sum": h.get("md5sum"),
            "cases.submitter_id": case_submitter,
            "cases.case_id": case_id
        })
    return pd.DataFrame(rows)

files_df = gdc_list_files(filters, fields_str)
if files_df.empty:
    raise RuntimeError("Nenhum arquivo CNV open-access encontrado. Verifique PROJECT_ID/DATA_CATEGORY.")

print(f"[OK] arquivos CNV encontrados: {len(files_df)}")
print(f"[INFO] % sem cases.submitter_id: {files_df['cases.submitter_id'].isna().mean():.3f}")

files_df_path = os.path.join(RAW_DIR, "gdc_cnv_files.tsv")
files_df.to_csv(files_df_path, sep="\t", index=False)
export_df_stats(files_df, "gdc_cnv_files")

files_df.head()


In [ ]:

# ================================
# 4) DOWNLOAD (with MD5 verification)
# ================================
def download_gdc_file(file_id: str, file_name: str, md5_expected: str | None, out_dir: str = DOWNLOAD_DIR) -> str:
    out_path = os.path.join(out_dir, file_name)
    if os.path.exists(out_path) and os.path.getsize(out_path) > 0:
        if md5_expected:
            md5_now = md5_file(out_path)
            if md5_now.lower() == str(md5_expected).lower():
                return out_path
            else:
                print(f"[WARN] md5 divergente para {file_name}. Rebaixando…")
                os.remove(out_path)

    url = f"{GDC_DATA_URL}/{file_id}"
    with requests.get(url, stream=True, timeout=300) as r:
        r.raise_for_status()
        with open(out_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=1<<20):
                if chunk:
                    f.write(chunk)

    if md5_expected:
        md5_now = md5_file(out_path)
        if md5_now.lower() != str(md5_expected).lower():
            raise RuntimeError(f"md5 inválido: {file_name} (expected={md5_expected}, got={md5_now})")
    return out_path

paths = []
for _, row in tqdm(files_df.iterrows(), total=len(files_df), desc="Downloading"):
    paths.append(download_gdc_file(row["file_id"], row["file_name"], row.get("md5sum")))

print(f"[OK] Downloads em: {DOWNLOAD_DIR}")
print(f"[OK] Arquivos baixados: {len(paths)}")


In [ ]:

# ==========================================
# 5) FILE METADATA (JOIN TABLE) — PUBLICÁVEL
# ==========================================
file_metadata = files_df[["file_id","file_name","md5sum","file_size","data_type","data_format","cases.submitter_id"]].copy()
file_metadata = file_metadata.rename(columns={"cases.submitter_id":"Participant_ID"})
file_metadata["Participant_ID"] = file_metadata["Participant_ID"].astype("string")

meta_path = os.path.join(PROC_DIR, "file_metadata.tsv")
file_metadata.to_csv(meta_path, sep="\t", index=False)
export_df_stats(file_metadata, "file_metadata")

file_metadata.head()


In [ ]:

# =====================================
# 6) LOAD + NORMALIZATION + QC (CANÔNICO)
# =====================================
COLMAP_CANDIDATES = {
    "Chromosome": ["Chromosome","chromosome","CHR","chr","Chrom","chrom"],
    "Start": ["Start","start","loc.start","start_pos","POS_START","Begin","start_position"],
    "End": ["End","end","loc.end","end_pos","POS_END","End_Position","end_position"],
    "Segment_Mean": ["Segment_Mean","segment_mean","SegmentMean","seg.mean","mean","value"]
}

def normalize_chr(x):
    if pd.isna(x):
        return pd.NA
    s = str(x).strip()
    s = s.replace("chr","").replace("CHR","").replace("Chr","")
    s = s.strip()
    if s in ["M","MT","m","mt"]:
        return pd.NA
    return s

VALID_CHR = set([str(i) for i in range(1,23)] + ["X","Y"])

def read_cnv_file(path: str) -> pd.DataFrame:
    try:
        df = pd.read_csv(path, sep="\t", low_memory=False)
    except Exception:
        df = pd.read_csv(path, sep=r"\s+", engine="python", low_memory=False)

    out = {}
    for std, cands in COLMAP_CANDIDATES.items():
        c = safe_col(df, cands)
        out[std] = df[c] if c is not None else pd.Series([pd.NA]*len(df))
    return pd.DataFrame(out)

meta_index = file_metadata.set_index("file_name")[["file_id","Participant_ID"]].to_dict(orient="index")

dfs = []
skipped = 0
for fname in tqdm(sorted(os.listdir(DOWNLOAD_DIR)), desc="Parsing CNV files"):
    fpath = os.path.join(DOWNLOAD_DIR, fname)
    if not os.path.isfile(fpath):
        continue
    if fname.endswith(".gz"):
        print(f"[WARN] Arquivo .gz não processado automaticamente: {fname}")
        skipped += 1
        continue

    df0 = read_cnv_file(fpath)
    m = meta_index.get(fname, None)
    df0["source_file"] = fname
    df0["file_id"] = m["file_id"] if m else pd.NA
    df0["Participant_ID"] = m["Participant_ID"] if m else pd.NA
    dfs.append(df0)

df_all = pd.concat(dfs, ignore_index=True)
print(f"[OK] Segmentos carregados: {len(df_all):,} | arquivos ignorados: {skipped}")

df_all["Chromosome"] = df_all["Chromosome"].map(normalize_chr).astype("string")
df_all["Start"] = pd.to_numeric(df_all["Start"], errors="coerce")
df_all["End"]   = pd.to_numeric(df_all["End"], errors="coerce")
df_all["Segment_Mean"] = pd.to_numeric(df_all["Segment_Mean"], errors="coerce")

df_all = df_all[df_all["Chromosome"].isin(VALID_CHR)]
df_all = df_all[df_all["Start"].notna() & df_all["End"].notna()]
df_all = df_all[df_all["End"] > df_all["Start"]]
df_all = df_all[df_all["Participant_ID"].notna()]

df_cnvs_final = df_all.copy()
df_cnvs_final["__JOIN_ID__"] = df_cnvs_final["Participant_ID"].astype("string")

parq_path = os.path.join(PROC_DIR, "cnv_segments.parquet")
tsv_path  = os.path.join(PROC_DIR, "combined_cnvs.txt")
df_cnvs_final.to_parquet(parq_path, index=False)
df_cnvs_final.to_csv(tsv_path, sep="\t", index=False)

export_df_stats(df_cnvs_final, "df_cnvs_final")

df_cnvs_final.head()


In [ ]:

# ==========================================
# 7) CNA CLASSIFICATION (Segment_Mean) — SAFE
# ==========================================
def classify_segment_mean(x: float) -> str:
    if pd.isna(x):
        return "Uncertain / margin"
    if x <= THRESHOLDS["LOSS_HOMDEL"]:
        return "Homozygous deletion (0 copies)"
    if x < THRESHOLDS["LOSS_1COPY_MAX"]:
        return "Loss (1 copy)"
    if THRESHOLDS["NEUTRAL_MIN"] <= x <= THRESHOLDS["NEUTRAL_MAX"]:
        return "Neutral (2 copies)"
    if x >= THRESHOLDS["GAIN_AMP"]:
        return "High-level amplification (≥4 copies)"
    if x > THRESHOLDS["GAIN_1COPY_MIN"]:
        return "Gain (3 copies)"
    return "Uncertain / margin"

df_cnvs_final["CNV_Type_Ajustado"] = df_cnvs_final["Segment_Mean"].map(classify_segment_mean).astype("string")

classified_path = os.path.join(PROC_DIR, "combined_cnvs_classified_adjusted.tsv")
df_cnvs_final.to_csv(classified_path, sep="\t", index=False)

export_df_stats(df_cnvs_final, "df_cnvs_final__classified")
df_cnvs_final["CNV_Type_Ajustado"].value_counts(dropna=False)


In [ ]:

# ====================================================
# 8) FILTER “HIGH-CONFIDENCE CNAs” (semântica correta)
# ====================================================
REMOVE_CLASSES = {"Cópia Neutra (2 Cópias)", "Indeterminado/Margem de Incerteza"}
df_cnvs_hc = df_cnvs_final[~df_cnvs_final["CNV_Type_Ajustado"].isin(REMOVE_CLASSES)].copy()

hc_path = os.path.join(PROC_DIR, "combined_cnvs_filtrado_somatica.txt")
df_cnvs_hc.to_csv(hc_path, sep="\t", index=False)

export_df_stats(df_cnvs_hc, "df_cnvs_high_confidence")
df_cnvs_hc.head()


# 8.5) RECURRENCE: exact SegmentID (chr:start-end) — “by patient” vs “by occurrence”

> **Why this step exists:** exact `SegmentID` is *fragile* (breakpoints vary across patients), so it typically underestimates true biological recurrence.
> Still, it can be useful for comparisons with “baseline” analyses and for discussion in the manuscript.



In [ ]:
import os
import pandas as pd
import numpy as np

# ==========================================
# 8.5) RECURRÊNCIA: SegmentID exato (chr:start-end)
#   - Por PACIENTE: % de pacientes com a CNV exata
#   - Por OCORRÊNCIA: % das linhas/segmentos (equivalente ao método antigo)
# ==========================================

if "df_cnvs_hc" not in globals() or not isinstance(df_cnvs_hc, pd.DataFrame) or len(df_cnvs_hc) == 0:
    raise RuntimeError("df_cnvs_hc não encontrado. Rode a etapa 8) FILTRO 'HIGH-CONFIDENCE CNAs' antes.")

df_seg = df_cnvs_hc.copy()

# Colunas canônicas esperadas no V4
REQ = ["Participant_ID", "Chromosome", "Start", "End", "CNV_Type_Ajustado"]
missing = [c for c in REQ if c not in df_seg.columns]
if missing:
    raise RuntimeError(f"Faltam colunas para SegmentID exato: {missing}. Disponíveis: {list(df_seg.columns)}")

# Tipos / limpeza mínima
df_seg["Chromosome"] = df_seg["Chromosome"].astype(str)
df_seg["Start"] = pd.to_numeric(df_seg["Start"], errors="coerce").astype("Int64")
df_seg["End"]   = pd.to_numeric(df_seg["End"],   errors="coerce").astype("Int64")
df_seg = df_seg.dropna(subset=["Participant_ID", "Chromosome", "Start", "End"])

# SegmentID exato
df_seg["SegmentID_exact"] = (
    df_seg["Chromosome"] + ":" +
    df_seg["Start"].astype(str) + "-" +
    df_seg["End"].astype(str)
)

N_patients = df_seg["Participant_ID"].nunique()
if N_patients == 0:
    raise RuntimeError("N_patients=0 após limpeza. Verifique Participant_ID / filtros anteriores.")

print(f"[INFO] SegmentID exact — N pacientes distintos: {N_patients:,}")
print(f"[INFO] SegmentID exact — N linhas (HC): {len(df_seg):,}")

# -------------------------
# A) Por PACIENTE (coorte)
# -------------------------
df_uniq = df_seg[["Participant_ID", "SegmentID_exact", "CNV_Type_Ajustado"]].drop_duplicates()

seg_by_patient = (
    df_uniq
    .groupby(["SegmentID_exact", "CNV_Type_Ajustado"])["Participant_ID"]
    .nunique()
    .reset_index(name="n_patients")
)
seg_by_patient["pct_patients"] = (seg_by_patient["n_patients"] / N_patients) * 100
seg_by_patient = seg_by_patient.sort_values(["n_patients", "pct_patients"], ascending=False)

# -------------------------
# B) Por OCORRÊNCIA (linhas)
# -------------------------
seg_by_rows = (
    df_seg
    .groupby(["SegmentID_exact", "CNV_Type_Ajustado"])
    .size()
    .reset_index(name="count_rows")
)
total_rows = int(seg_by_rows["count_rows"].sum())
seg_by_rows["pct_rows"] = (seg_by_rows["count_rows"] / total_rows) * 100
seg_by_rows = seg_by_rows.sort_values("count_rows", ascending=False)

# -------------------------
# C) Comparativo lado-a-lado
# -------------------------
seg_compare = (
    seg_by_patient
    .merge(seg_by_rows, on=["SegmentID_exact", "CNV_Type_Ajustado"], how="left")
    .sort_values(["n_patients", "count_rows"], ascending=False)
)

display(seg_compare.head(15))

# -------------------------
# Export (TSV)
# -------------------------
RES_DIR = globals().get("RES_DIR", globals().get("RESULTS_DIR", "outputs/segmentid_exact/results"))
os.makedirs(RES_DIR, exist_ok=True)

p_patient = os.path.join(RES_DIR, "cnv_recurrence_by_segmentid_exact__by_patient.tsv")
p_rows    = os.path.join(RES_DIR, "cnv_recurrence_by_segmentid_exact__by_rows.tsv")
p_cmp     = os.path.join(RES_DIR, "cnv_recurrence_by_segmentid_exact__compare.tsv")

seg_by_patient.to_csv(p_patient, sep="\t", index=False)
seg_by_rows.to_csv(p_rows, sep="\t", index=False)
seg_compare.to_csv(p_cmp, sep="\t", index=False)

print("[OK] Exportados:")
print(" -", p_patient)
print(" -", p_rows)
print(" -", p_cmp)


In [ ]:
# ==========================================
# 9) RECURRÊNCIA: CITOBANDA (hg38, overlap) — MEMORY SAFE + PUBLICÁVEL
# ==========================================
import os, gzip, requests
import numpy as np
import pandas as pd

# pyranges
try:
    import pyranges as pr
except Exception as e:
    raise RuntimeError("pyranges não está disponível. Instale: !pip -q install pyranges") from e

# Diretórios (padrão do notebook)
RAW_DIR  = globals().get("RAW_DIR",  os.path.join(BASE_OUT_DIR, "raw"))
PROC_DIR = globals().get("PROC_DIR", os.path.join(BASE_OUT_DIR, "processed"))
RES_DIR  = globals().get("RES_DIR",  os.path.join(BASE_OUT_DIR, "results"))
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROC_DIR, exist_ok=True)
os.makedirs(RES_DIR, exist_ok=True)

CYTO_URL = "https://hgdownload.soe.ucsc.edu/goldenPath/hg38/database/cytoBand.txt.gz"
cyto_gz = os.path.join(RAW_DIR, "cytoBand_hg38.txt.gz")
cyto_tsv = os.path.join(PROC_DIR, "cytoBand_hg38.tsv")

# ----------------------------
# 1) Baixar + parse (cacheado)
# ----------------------------
if not os.path.exists(cyto_tsv):
    if not os.path.exists(cyto_gz):
        r = requests.get(CYTO_URL, timeout=120)
        r.raise_for_status()
        with open(cyto_gz, "wb") as f:
            f.write(r.content)
        print(f"[OK] baixado: {cyto_gz}")

    rows = []
    with gzip.open(cyto_gz, "rt") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) >= 5:
                chrom, start, end, name, stain = parts[:5]
                chrom = chrom.replace("chr", "")
                if chrom in VALID_CHR:
                    rows.append([chrom, int(start), int(end), name, stain])

    cyto_df = pd.DataFrame(rows, columns=["Chromosome", "Start", "End", "Cytoband", "Stain"])
    cyto_df.to_csv(cyto_tsv, sep="\t", index=False)
else:
    cyto_df = pd.read_csv(cyto_tsv, sep="\t")

export_df_stats(cyto_df, "cytoBand_hg38")

# -----------------------------------
# 2) Validar df_cnvs_hc e padronizar
# -----------------------------------
cnv_cols = ["Chromosome", "Start", "End", "Participant_ID", "CNV_Type_Ajustado"]
missing = [c for c in cnv_cols if c not in df_cnvs_hc.columns]
if missing:
    raise RuntimeError(f"df_cnvs_hc está sem colunas obrigatórias para citobanda: {missing}")

df_cnvs_hc = df_cnvs_hc.copy()
df_cnvs_hc["Chromosome"] = df_cnvs_hc["Chromosome"].astype(str).str.replace("chr", "", regex=False)
df_cnvs_hc["Participant_ID"] = df_cnvs_hc["Participant_ID"].astype(str)
df_cnvs_hc["CNV_Type_Ajustado"] = df_cnvs_hc["CNV_Type_Ajustado"].astype(str)
df_cnvs_hc["Start"] = pd.to_numeric(df_cnvs_hc["Start"], errors="coerce")
df_cnvs_hc["End"]   = pd.to_numeric(df_cnvs_hc["End"], errors="coerce")
df_cnvs_hc = df_cnvs_hc.dropna(subset=["Start","End"])
df_cnvs_hc["Start"] = df_cnvs_hc["Start"].astype(int)
df_cnvs_hc["End"]   = df_cnvs_hc["End"].astype(int)
df_cnvs_hc = df_cnvs_hc[df_cnvs_hc["Chromosome"].isin(VALID_CHR)].copy()

seg_key = ["Participant_ID", "Chromosome", "Start", "End", "CNV_Type_Ajustado"]

def chr_sort_key(x: str):
    if x.isdigit():
        return (0, int(x))
    if x == "X":
        return (1, 23)
    if x == "Y":
        return (1, 24)
    return (2, 999)

chroms_to_process = sorted(df_cnvs_hc["Chromosome"].unique().tolist(), key=chr_sort_key)
best_chunks = []

# ----------------------------
# 3) Join CNV × cytoband por cromossomo (memory safe)
# ----------------------------
for chrom in chroms_to_process:
    df_chr = df_cnvs_hc[df_cnvs_hc["Chromosome"] == chrom][cnv_cols].copy()
    if df_chr.empty:
        continue

    cyto_chr = cyto_df[cyto_df["Chromosome"] == chrom][["Chromosome", "Start", "End", "Cytoband"]].copy()
    if cyto_chr.empty:
        tmp = df_chr.copy()
        tmp["Cytoband"] = np.nan
        tmp["overlap_bp"] = 0
        best_chunks.append(tmp[seg_key + ["Cytoband", "overlap_bp"]])
        continue

    cnv_pr = pr.PyRanges(df_chr)
    cyto_pr = pr.PyRanges(cyto_chr)

    ov = cnv_pr.join(cyto_pr, suffix="_cyto")
    ov_df = ov.df
    if ov_df.empty:
        tmp = df_chr.copy()
        tmp["Cytoband"] = np.nan
        tmp["overlap_bp"] = 0
        best_chunks.append(tmp[seg_key + ["Cytoband", "overlap_bp"]])
        continue

    cyto_col = "Cytoband_cyto" if "Cytoband_cyto" in ov_df.columns else "Cytoband"
    ov_df = ov_df[seg_key + ["Start_cyto", "End_cyto", cyto_col]].copy()

    ov_df["overlap_bp"] = (
        np.minimum(ov_df["End"], ov_df["End_cyto"]) - np.maximum(ov_df["Start"], ov_df["Start_cyto"])
    ).clip(lower=0).astype("int32")

    idx = ov_df.groupby(seg_key, observed=True)["overlap_bp"].idxmax()
    best_chr = ov_df.loc[idx, seg_key + [cyto_col, "overlap_bp"]].copy()
    best_chr = best_chr.rename(columns={cyto_col: "Cytoband"})
    best_chunks.append(best_chr)

best = pd.concat(best_chunks, ignore_index=True) if best_chunks else pd.DataFrame(columns=seg_key + ["Cytoband", "overlap_bp"])

# ----------------------------
# 4) Merge, export e recorrência publicável
# ----------------------------
df_cnvs_hc_annot = df_cnvs_hc.merge(best, on=seg_key, how="left")

annot_path = os.path.join(PROC_DIR, "cnv_segments_high_confidence_cytoband.tsv")
df_cnvs_hc_annot.to_csv(annot_path, sep="\t", index=False)
export_df_stats(df_cnvs_hc_annot, "df_cnvs_hc__cytoband")

# edges paciente↔citobanda (presença)
edges_cyto = df_cnvs_hc_annot[["Participant_ID","Chromosome","Cytoband","CNV_Type_Ajustado","overlap_bp"]].copy()
edges_cyto["Participant_ID"] = edges_cyto["Participant_ID"].astype(str)
edges_cyto["Chromosome"] = edges_cyto["Chromosome"].astype(str)
edges_cyto["CNV_Type_Ajustado"] = edges_cyto["CNV_Type_Ajustado"].astype(str)
edges_cyto = edges_cyto.dropna(subset=["Cytoband"])
edges_cyto = edges_cyto.drop_duplicates(["Participant_ID","Chromosome","Cytoband","CNV_Type_Ajustado"])

edges_path = os.path.join(PROC_DIR, "cnv_patient_cytoband_overlaps.tsv")
edges_cyto.to_csv(edges_path, sep="\t", index=False)

# recorrência por citobanda (pacientes únicos)
rec_cyto = (edges_cyto
    .groupby(["Chromosome","Cytoband","CNV_Type_Ajustado"], observed=True)["Participant_ID"]
    .nunique()
    .reset_index(name="n_patients")
    .sort_values(["Chromosome","n_patients"], ascending=[True, False])
)
out_path = os.path.join(RES_DIR, "cnv_recurrence_by_cytoband.tsv")
rec_cyto.to_csv(out_path, sep="\t", index=False)
export_df_stats(rec_cyto, "cnv_recurrence_by_cytoband")

print(f"[OK] anotado: {annot_path}")
print(f"[OK] edges paciente↔citobanda: {edges_path}")
print(f"[OK] recorrência citobanda: {out_path}")

rec_cyto.head(10)


In [ ]:
# ==========================================
# 10) RECURRÊNCIA: BINS FIXOS (1Mb, hg38, overlap) — PUBLICÁVEL + COMPATÍVEL COM PLOTS
# ==========================================
import os, requests
import numpy as np
import pandas as pd

try:
    import pyranges as pr
except Exception as e:
    raise RuntimeError("pyranges não está disponível. Instale: !pip -q install pyranges") from e

# Diretórios (padrão do notebook)
PROC_DIR = globals().get("PROC_DIR", os.path.join(BASE_OUT_DIR, "processed"))
RES_DIR  = globals().get("RES_DIR",  os.path.join(BASE_OUT_DIR, "results"))
LOG_DIR  = globals().get("LOG_DIR",  os.path.join(BASE_OUT_DIR, "logs"))
os.makedirs(PROC_DIR, exist_ok=True)
os.makedirs(RES_DIR, exist_ok=True)

CHROMSIZES_URL = "https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes"
chromsizes_path = os.path.join(PROC_DIR, "hg38.chrom.sizes")

BIN_SIZE = int(globals().get("BIN_SIZE", 1_000_000))
MIN_OVERLAP_BP = int(globals().get("MIN_OVERLAP_BP", 1))  # para presença em bin

# ----------------------------
# 1) Baixar chrom.sizes (cacheado)
# ----------------------------
if not os.path.exists(chromsizes_path):
    r = requests.get(CHROMSIZES_URL, timeout=120)
    r.raise_for_status()
    with open(chromsizes_path, "wb") as f:
        f.write(r.content)
    print(f"[OK] chrom.sizes → {chromsizes_path}")
else:
    print(f"[OK] chrom.sizes (cache) → {chromsizes_path}")

# ----------------------------
# 2) Ler tamanhos e filtrar cromossomos
# ----------------------------
sizes = []
with open(chromsizes_path, "rt") as f:
    for line in f:
        chrom, size = line.strip().split("\t")[:2]
        chrom = chrom.replace("chr", "")
        if chrom in VALID_CHR:
            sizes.append((chrom, int(size)))

sizes_df = pd.DataFrame(sizes, columns=["Chromosome", "ChromSize"])
export_df_stats(sizes_df, "hg38_chrom_sizes_filtered")

# ----------------------------
# 3) Gerar bins 1Mb (BinStart/BinEnd) + BinID
# ----------------------------
bin_rows = []
for chrom, chrom_size in sizes:
    for bs in range(0, chrom_size, BIN_SIZE):
        be = min(bs + BIN_SIZE, chrom_size)
        bin_rows.append((chrom, bs, be))

bins_df = pd.DataFrame(bin_rows, columns=["Chromosome", "BinStart", "BinEnd"])
bins_df["BinID"] = (
    bins_df["Chromosome"].astype(str)
    + ":"
    + bins_df["BinStart"].astype(str)
    + "-"
    + bins_df["BinEnd"].astype(str)
)

# colunas Start/End para o PyRanges (mantendo BinStart/BinEnd para export/plots)
bins_pr_df = bins_df.rename(columns={"BinStart": "Start", "BinEnd": "End"})[["Chromosome","Start","End","BinID"]].copy()
bins_pr_df["Start"] = bins_pr_df["Start"].astype(int)
bins_pr_df["End"]   = bins_pr_df["End"].astype(int)

export_df_stats(bins_df, f"hg38_bins_{BIN_SIZE}")

# ----------------------------
# 4) Criar PyRanges a partir de DataFrame
# ----------------------------
bins_pr = pr.PyRanges(bins_pr_df)

# CNVs high-confidence
cnv_cols = ["Chromosome", "Start", "End", "Participant_ID", "CNV_Type_Ajustado"]
missing = [c for c in cnv_cols if c not in df_cnvs_hc.columns]
if missing:
    raise RuntimeError(f"df_cnvs_hc está sem colunas obrigatórias para bins: {missing}")

df_cnv_bins = df_cnvs_hc[cnv_cols].copy()
df_cnv_bins["Chromosome"] = df_cnv_bins["Chromosome"].astype(str).str.replace("chr","", regex=False)
df_cnv_bins["Participant_ID"] = df_cnv_bins["Participant_ID"].astype(str)
df_cnv_bins["CNV_Type_Ajustado"] = df_cnv_bins["CNV_Type_Ajustado"].astype(str)

df_cnv_bins["Start"] = pd.to_numeric(df_cnv_bins["Start"], errors="coerce")
df_cnv_bins["End"]   = pd.to_numeric(df_cnv_bins["End"], errors="coerce")
df_cnv_bins = df_cnv_bins.dropna(subset=["Start","End"])
df_cnv_bins["Start"] = df_cnv_bins["Start"].astype(int)
df_cnv_bins["End"]   = df_cnv_bins["End"].astype(int)

df_cnv_bins = df_cnv_bins[df_cnv_bins["Chromosome"].isin(VALID_CHR)].copy()

cnv_pr = pr.PyRanges(df_cnv_bins)

# ----------------------------
# 5) Overlap CNV × bins (edges paciente↔bin) + recorrência
# ----------------------------
ov = cnv_pr.join(bins_pr, suffix="_bin")
ov_df = ov.df.copy()

if ov_df.empty:
    raise RuntimeError("Overlap CNV×bins retornou vazio. Verifique cromossomos/Start/End.")

# detectar nomes com suffix
binid_col = "BinID_bin" if "BinID_bin" in ov_df.columns else "BinID"
start_bin_col = "Start_bin" if "Start_bin" in ov_df.columns else ("Start_bin" if "Start_bin" in ov_df.columns else None)
end_bin_col   = "End_bin"   if "End_bin" in ov_df.columns else ("End_bin"   if "End_bin"   in ov_df.columns else None)

# Em algumas versões, as colunas podem vir como Start_bin / End_bin mesmo (normal)
if start_bin_col is None or end_bin_col is None:
    # fallback: tenta 'Start_bin'/'End_bin' padrões do pyranges
    cand = [c for c in ov_df.columns if c.lower() in ("start_bin","start_b","start_right")]
    cand2 = [c for c in ov_df.columns if c.lower() in ("end_bin","end_b","end_right")]
    if cand and cand2:
        start_bin_col, end_bin_col = cand[0], cand2[0]
    else:
        raise RuntimeError(f"Não encontrei colunas de bin Start/End no join. Colunas: {list(ov_df.columns)}")

# overlap_bp para filtro
ov_df["overlap_bp"] = (
    np.minimum(ov_df["End"], ov_df[end_bin_col]) - np.maximum(ov_df["Start"], ov_df[start_bin_col])
).clip(lower=0).astype("int32")

ov_df = ov_df[ov_df["overlap_bp"] >= MIN_OVERLAP_BP].copy()

edges = ov_df[["Participant_ID","Chromosome", start_bin_col, end_bin_col, binid_col, "CNV_Type_Ajustado", "overlap_bp"]].copy()
edges = edges.rename(columns={
    start_bin_col: "BinStart",
    end_bin_col: "BinEnd",
    binid_col: "BinID"
})
edges["Participant_ID"] = edges["Participant_ID"].astype(str)
edges["Chromosome"] = edges["Chromosome"].astype(str).str.replace("chr","", regex=False)
edges["CNV_Type_Ajustado"] = edges["CNV_Type_Ajustado"].astype(str)
edges["BinStart"] = edges["BinStart"].astype(int)
edges["BinEnd"]   = edges["BinEnd"].astype(int)
edges["BinID"]    = edges["BinID"].astype(str)

# dedup: presença por paciente/bin/tipo
edges_uniq = edges.drop_duplicates(["Participant_ID","BinID","CNV_Type_Ajustado"]).copy()

edges_path = os.path.join(PROC_DIR, f"cnv_patient_bin_overlaps_{BIN_SIZE//1_000_000}Mb.tsv")
edges_uniq.to_csv(edges_path, sep="\t", index=False)
export_df_stats(edges_uniq, f"cnv_patient_bin_overlaps_{BIN_SIZE}")

# recorrência: pacientes únicos por bin/tipo
rec_bins = (edges_uniq
    .groupby(["Chromosome","BinStart","BinEnd","BinID","CNV_Type_Ajustado"], observed=True)["Participant_ID"]
    .nunique()
    .reset_index(name="n_patients")
    .sort_values(["Chromosome","n_patients"], ascending=[True, False])
)

out_path = os.path.join(RES_DIR, f"cnv_recurrence_by_bins_{BIN_SIZE//1_000_000}Mb.tsv")
rec_bins.to_csv(out_path, sep="\t", index=False)
export_df_stats(rec_bins, f"cnv_recurrence_by_bins_{BIN_SIZE}")

print(f"[OK] edges paciente↔bin: {edges_path}")
print(f"[OK] recorrência por bins salva: {out_path}")
rec_bins.head(10)


In [ ]:

# ==========================================
# 11) BREAKPOINT FEATURES (por paciente)
# ==========================================
df = df_cnvs_hc.sort_values(["Participant_ID","Chromosome","Start","End"]).copy()

bp = pd.concat([
    df[["Participant_ID","Chromosome","Start"]].rename(columns={"Start":"pos"}),
    df[["Participant_ID","Chromosome","End"]].rename(columns={"End":"pos"})
], ignore_index=True)

bp["pos"] = pd.to_numeric(bp["pos"], errors="coerce")
bp = bp.dropna(subset=["pos"])

bp_total = bp.groupby("Participant_ID").size().rename("BP_total").reset_index()
bp_unique = bp.groupby("Participant_ID")["pos"].nunique().rename("BP_unique").reset_index()

chr_counts = bp.groupby(["Participant_ID","Chromosome"]).size().reset_index(name="n_bp")

def shannon_entropy_norm(x):
    x = np.asarray(x, dtype=float)
    x = x[x>0]
    if len(x) <= 1:
        return 0.0
    p = x / x.sum()
    H = -(p * np.log(p)).sum()
    Hmax = np.log(len(p))
    return float(H / Hmax) if Hmax>0 else 0.0

def gini(x):
    x = np.asarray(x, dtype=float)
    x = x[x>=0]
    if len(x)==0:
        return np.nan
    if np.all(x==0):
        return 0.0
    x = np.sort(x)
    n = len(x)
    cumx = np.cumsum(x)
    return float((n + 1 - 2*np.sum(cumx)/cumx[-1]) / n)

ent = chr_counts.groupby("Participant_ID")["n_bp"].apply(shannon_entropy_norm).rename("BP_entropy_chr").reset_index()
gin = chr_counts.groupby("Participant_ID")["n_bp"].apply(gini).rename("BP_gini_chr").reset_index()

bp_metrics_df = bp_total.merge(bp_unique, on="Participant_ID").merge(ent, on="Participant_ID").merge(gin, on="Participant_ID")
bp_metrics_df["__JOIN_ID__"] = bp_metrics_df["Participant_ID"].astype("string")

export_df_stats(bp_metrics_df, "bp_metrics_df")
bp_metrics_path = os.path.join(RES_DIR, "breakpoint_metrics_per_patient.tsv")
bp_metrics_df.to_csv(bp_metrics_path, sep="\t", index=False)

bp_metrics_df.head()


In [ ]:

# ==========================================================
# 12) HOTSPOT FEATURES (por paciente, janelas 1Mb)
# ==========================================================
WINDOW_SIZE = BIN_SIZE
TOPK = 50

bp2 = bp.copy()
bp2["BinStart"] = (bp2["pos"] // WINDOW_SIZE) * WINDOW_SIZE
win_counts = bp2.groupby(["Participant_ID","Chromosome","BinStart"]).size().reset_index(name="bp_count")

p95 = np.percentile(win_counts["bp_count"], 95) if len(win_counts) else np.nan

hot_p95 = (
    win_counts.assign(is_hot = win_counts["bp_count"] >= p95)
    .groupby("Participant_ID")["is_hot"].sum()
    .rename("HotspotCount_P95")
    .reset_index()
)

def topk_sum(s, k=TOPK):
    v = np.sort(np.asarray(s, dtype=float))[::-1]
    return float(v[:k].sum()) if len(v) else 0.0

hot_topk = win_counts.groupby("Participant_ID")["bp_count"].apply(lambda s: topk_sum(s, TOPK)).rename("HotspotScore_topK").reset_index()
hot_ent = win_counts.groupby("Participant_ID")["bp_count"].apply(shannon_entropy_norm).rename("HotspotEntropy").reset_index()

hotspot_metrics_df = hot_p95.merge(hot_topk, on="Participant_ID").merge(hot_ent, on="Participant_ID")
hotspot_metrics_df["__JOIN_ID__"] = hotspot_metrics_df["Participant_ID"].astype("string")

export_df_stats(hotspot_metrics_df, "hotspot_metrics_df")
hotspot_path = os.path.join(RES_DIR, "hotspot_metrics_per_patient.tsv")
hotspot_metrics_df.to_csv(hotspot_path, sep="\t", index=False)

hotspot_metrics_df.head()


## Clinical / Survival block (LOCKDOWN)

Builds `os_df` **without imputation** from `clinical.tsv` and `follow_up.tsv` (when available).



In [ ]:
# =========================================================
# 13) OS LOCKDOWN (sem imputação) + STAGING (todas as colunas disponíveis)
#     + merge opcional de family_history / exposure / pathology_detail (se existirem)
# =========================================================
import glob
import re
import numpy as np
import pandas as pd

def _find_first(paths):
    for p in paths:
        hits = glob.glob(p, recursive=True)
        if hits:
            hits = sorted(list(dict.fromkeys(hits)), key=lambda x:(len(x), x))
            return hits[0]
    return None

# busca em /content (Colab) + /mnt/data (sandbox) + Google Drive
clinical_path = _find_first([
    "/content/clinical.tsv",
    "/content/**/clinical.tsv",
    "/content/**/clinical*.tsv",
    "/content/drive/MyDrive/**/clinical.tsv",
    "/mnt/data/clinical.tsv",
    "/mnt/data/**/clinical.tsv",
])

followup_path = _find_first([
    "/content/follow_up.tsv",
    "/content/**/follow_up.tsv",
    "/content/**/follow_up*.tsv",
    "/content/drive/MyDrive/**/follow_up.tsv",
    "/mnt/data/follow_up.tsv",
    "/mnt/data/**/follow_up.tsv",
])

family_path = _find_first([
    "/content/family_history.tsv",
    "/content/**/family_history.tsv",
    "/content/**/family_history*.tsv",
    "/content/drive/MyDrive/**/family_history.tsv",
    "/mnt/data/family_history.tsv",
    "/mnt/data/**/family_history.tsv",
])

exposure_path = _find_first([
    "/content/exposure.tsv",
    "/content/**/exposure.tsv",
    "/content/**/exposure*.tsv",
    "/content/drive/MyDrive/**/exposure.tsv",
    "/mnt/data/exposure.tsv",
    "/mnt/data/**/exposure.tsv",
])

pathology_detail_path = _find_first([
    "/content/pathology_detail.tsv",
    "/content/**/pathology_detail.tsv",
    "/content/**/pathology_detail*.tsv",
    "/content/drive/MyDrive/**/pathology_detail.tsv",
    "/mnt/data/pathology_detail.tsv",
    "/mnt/data/**/pathology_detail.tsv",
])

if clinical_path is None:
    raise FileNotFoundError("Não encontrei clinical.tsv. Faça upload no Colab (ou deixe em /mnt/data) e rode novamente.")

df_clin = pd.read_csv(clinical_path, sep="\t", low_memory=False)
print(f"[OK] clinical.tsv: {clinical_path} | shape={df_clin.shape}")

# follow_up é grande; ler só colunas necessárias
df_fu = None
if followup_path is not None:
    fu_cols = pd.read_csv(followup_path, sep="\t", nrows=0).columns.tolist()
    fu_id_col = next((c for c in ["cases.submitter_id","cases.case_id"] if c in fu_cols), None)
    fu_time_col = next((c for c in ["follow_ups.days_to_follow_up","follow_ups.days_to_last_follow_up","follow_ups.days_to_followup"] if c in fu_cols), None)
    if fu_id_col and fu_time_col:
        df_fu = pd.read_csv(followup_path, sep="\t", usecols=[fu_id_col, fu_time_col], low_memory=False)
        print(f"[OK] follow_up.tsv (usecols): {followup_path} | shape={df_fu.shape}")
    else:
        print("[WARN] follow_up.tsv encontrado, mas sem colunas esperadas de ID/tempo. Ignorando follow_up.")
else:
    print("[WARN] follow_up.tsv não encontrado.")

def pick_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

# ----------------------------
# Identificadores e campos de OS
# ----------------------------
id_col = pick_col(df_clin, ["cases.submitter_id","demographic.submitter_id","cases.case_id"])
if id_col is None:
    raise ValueError("Sem identificador (cases.submitter_id) no clinical.tsv.")

vital_col = pick_col(df_clin, ["demographic.vital_status","cases.vital_status"])
death_days_col = pick_col(df_clin, ["demographic.days_to_death","cases.diagnoses.days_to_death","diagnoses.days_to_death","cases.diagnoses.days_to_death"])
age_col = pick_col(df_clin, ["demographic.age_at_index","demographic.age_at_index".replace("_",".")])
sex_col = pick_col(df_clin, ["demographic.gender","demographic.sex","cases.gender","cases.sex"])

# ----------------------------
# Estadiamento (TODAS as colunas plausíveis)
# Observação: aqui só anexamos ao os_df; os filtros e análises são feitos depois.
# ----------------------------
stage_like = []
for c in df_clin.columns:
    lc = c.lower()
    if ("iss_stage" in lc) or ("r_iss" in lc) or ("riss" in lc) or ("revised" in lc and "iss" in lc):
        stage_like.append(c)
    elif re.search(r"(?:^|\.|_)(stage|staging)(?:$|_|\.)", lc):
        # inclui *stage* genérico (pode estar vazio; filtraremos depois)
        stage_like.append(c)

STAGE_COLUMNS_AVAILABLE = sorted(list(dict.fromkeys(stage_like)))

# ----------------------------
# Base OS
# ----------------------------
base = df_clin[[id_col]].copy().rename(columns={id_col:"Participant_ID"})
base["Participant_ID"] = base["Participant_ID"].astype(str)

vital = df_clin[vital_col].astype(str).str.strip().str.lower() if vital_col else pd.Series([pd.NA]*len(df_clin))
days_to_death = pd.to_numeric(df_clin[death_days_col], errors="coerce") if death_days_col else pd.Series([np.nan]*len(df_clin))

if age_col and age_col in df_clin.columns:
    base["age"] = pd.to_numeric(df_clin[age_col], errors="coerce")
else:
    base["age"] = np.nan

if sex_col and sex_col in df_clin.columns:
    base["sex"] = df_clin[sex_col]
else:
    base["sex"] = pd.NA

# adiciona colunas de estadiamento (raw)
if STAGE_COLUMNS_AVAILABLE:
    st = df_clin[[id_col] + STAGE_COLUMNS_AVAILABLE].copy()
    st = st.rename(columns={id_col:"Participant_ID"})
    st["Participant_ID"] = st["Participant_ID"].astype(str)

    # resolve duplicatas por Participant_ID pegando o primeiro valor não-nulo (determinístico)
    def _first_nonnull(s):
        s2 = s.dropna()
        if len(s2) == 0:
            return pd.NA
        return s2.iloc[0]

    agg_map = {c: _first_nonnull for c in STAGE_COLUMNS_AVAILABLE}
    st = st.groupby("Participant_ID", as_index=False).agg(agg_map)
    base = base.merge(st, on="Participant_ID", how="left")

base["event"] = ((vital=="dead") | days_to_death.notna()).astype(int)
base["time"]  = np.where(base["event"].eq(1), days_to_death, np.nan)

# follow-up para censura (sem imputação "inventada")
if df_fu is not None:
    fu_id_col = pick_col(df_fu, ["cases.submitter_id","cases.case_id"])
    fu_time_col = pick_col(df_fu, ["follow_ups.days_to_follow_up","follow_ups.days_to_last_follow_up","follow_ups.days_to_followup"])
    if fu_id_col and fu_time_col:
        fu_tmp = df_fu[[fu_id_col, fu_time_col]].copy().rename(columns={fu_id_col:"Participant_ID"})
        fu_tmp["Participant_ID"] = fu_tmp["Participant_ID"].astype(str)
        fu_tmp[fu_time_col] = pd.to_numeric(fu_tmp[fu_time_col], errors="coerce")
        fu_max = fu_tmp.groupby("Participant_ID")[fu_time_col].max().rename("fu_time_max").reset_index()
        base = base.merge(fu_max, on="Participant_ID", how="left")
    else:
        base["fu_time_max"] = np.nan
        print("[WARN] follow_up.tsv sem colunas esperadas.")
else:
    base["fu_time_max"] = np.nan

alive = base["event"].eq(0)
base.loc[alive, "time"] = base.loc[alive, "fu_time_max"]

# ----------------------------
# Family history (opcional): agrega por paciente
# ----------------------------
if family_path is not None:
    df_fh = pd.read_csv(family_path, sep="\t", low_memory=False)
    print(f"[OK] family_history.tsv: {family_path} | shape={df_fh.shape}")
    fh_id = pick_col(df_fh, ["cases.submitter_id","cases.case_id"])
    if fh_id:
        fh = df_fh.copy()
        fh = fh.rename(columns={fh_id:"Participant_ID"})
        fh["Participant_ID"] = fh["Participant_ID"].astype(str)

        # features simples e determinísticas (sem inventar)
        if "family_histories.relatives_with_cancer_history_count" in fh.columns:
            fh["fh_cancer_count"] = pd.to_numeric(fh["family_histories.relatives_with_cancer_history_count"], errors="coerce")
        else:
            fh["fh_cancer_count"] = np.nan

        if "family_histories.relative_with_cancer_history" in fh.columns:
            v = fh["family_histories.relative_with_cancer_history"].astype(str).str.lower().str.strip()
            fh["fh_any_cancer_history"] = v.isin(["yes","true","1"]).astype(int)
        else:
            fh["fh_any_cancer_history"] = np.nan

        fh_agg = fh.groupby("Participant_ID", as_index=False).agg({
            "fh_cancer_count": "max",
            "fh_any_cancer_history": "max"
        })
        base = base.merge(fh_agg, on="Participant_ID", how="left")
else:
    print("[INFO] family_history.tsv não encontrado (ok).")

# Exposure/pathology_detail existem às vezes vazios; apenas logamos
if exposure_path is not None:
    try:
        df_ex = pd.read_csv(exposure_path, sep="\t", low_memory=False)
        print(f"[OK] exposure.tsv: {exposure_path} | shape={df_ex.shape} (não usado no OS por padrão)")
    except Exception:
        print("[WARN] exposure.tsv encontrado, mas falhou leitura (ignorado).")

if pathology_detail_path is not None:
    try:
        df_pd = pd.read_csv(pathology_detail_path, sep="\t", low_memory=False)
        print(f"[OK] pathology_detail.tsv: {pathology_detail_path} | shape={df_pd.shape} (não usado no OS por padrão)")
    except Exception:
        print("[WARN] pathology_detail.tsv encontrado, mas falhou leitura (ignorado).")

# ----------------------------
# Final OS
# ----------------------------
os_df = base.copy()
os_df["time"] = pd.to_numeric(os_df["time"], errors="coerce")
os_df = os_df[(os_df["time"].notna()) & (os_df["time"] >= 0)]
os_df["Participant_ID"] = os_df["Participant_ID"].astype(str)
os_df["__JOIN_ID__"] = os_df["Participant_ID"].astype("string")

export_df_stats(os_df, "os_df")
print(f"[OK] os_df: n={len(os_df)} | event rate={os_df['event'].mean():.3f}")
print(f"[OK] Estadiamento disponíveis (clinical): {len(STAGE_COLUMNS_AVAILABLE)} colunas")
os_df.head()


In [ ]:
# =========================================================
# (REFAZER INPUT CLÍNICO) OS/CLINICAL LOCKDOWN — auditável
# Cria os_df paciente-nível com: age, demographic.gender, time, event, ISS etc.
# =========================================================
import os, numpy as np, pandas as pd

def _find_first_existing(candidates):
    for p in candidates:
        if p and os.path.exists(p):
            return p
    return None

# ---- localizar arquivos (prioriza /mnt/data) ----
clinical_path = _find_first_existing([
    "/mnt/data/clinical.tsv",
    "/content/clinical.tsv",
    os.path.join(os.getcwd(), "clinical.tsv"),
])
followup_path = _find_first_existing([
    "/mnt/data/follow_up.tsv",
    "/content/follow_up.tsv",
    os.path.join(os.getcwd(), "follow_up.tsv"),
])

if clinical_path is None:
    raise FileNotFoundError("Não achei clinical.tsv em /mnt/data nem /content.")
if followup_path is None:
    raise FileNotFoundError("Não achei follow_up.tsv em /mnt/data nem /content.")

# ---- diretórios de saída ----
OUT_DIR = globals().get("OUT_DIR") or os.path.join(os.getcwd(), "outputs")
RES_DIR = globals().get("RES_DIR") or globals().get("RESULTS_DIR") or os.path.join(OUT_DIR, "results")
LOG_DIR = globals().get("LOG_DIR") or os.path.join(OUT_DIR, "logs")
os.makedirs(RES_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# ---- ler header do clinical para capturar todos os stage cols ----
with open(clinical_path, "r") as f:
    header = f.readline().strip().split("\t")

stage_cols = [c for c in header if c.startswith("diagnoses.") and c.endswith("_stage")]

essential_cols = [
    "cases.submitter_id",
    "demographic.age_at_index",
    "demographic.gender",
    "demographic.vital_status",
    "demographic.days_to_death",
    "diagnoses.days_to_last_follow_up",
]
usecols = list(dict.fromkeys(essential_cols + stage_cols))

clinical_df = pd.read_csv(clinical_path, sep="\t", usecols=usecols, low_memory=False)

# ---- limpeza de NA-like (ex.: '--') sem inventar dado ----
na_like = {"'--", "--", "NA", "N/A", "NaN", "nan", "", "None", "null", "NULL"}

def _clean_obj(s):
    # mantém strings reais, troca placeholders por NaN
    s = s.astype(str).replace(list(na_like), np.nan)
    return s

for c in clinical_df.columns:
    if clinical_df[c].dtype == object:
        clinical_df[c] = _clean_obj(clinical_df[c])

clinical_df["demographic.age_at_index"] = pd.to_numeric(clinical_df["demographic.age_at_index"], errors="coerce")
clinical_df["demographic.days_to_death"] = pd.to_numeric(clinical_df["demographic.days_to_death"], errors="coerce")
clinical_df["diagnoses.days_to_last_follow_up"] = pd.to_numeric(clinical_df["diagnoses.days_to_last_follow_up"], errors="coerce")

# ---- agregação paciente-nível (995 pacientes) ----
def first_valid(x):
    x = x.dropna()
    return x.iloc[0] if len(x) else np.nan

agg = {
    "demographic.age_at_index": "first",
    "demographic.gender": first_valid,
    "demographic.vital_status": first_valid,
    "demographic.days_to_death": "max",
    "diagnoses.days_to_last_follow_up": "max",
}
for sc in stage_cols:
    agg[sc] = first_valid

pat_df = clinical_df.groupby("cases.submitter_id", as_index=False).agg(agg)
pat_df = pat_df.rename(columns={"cases.submitter_id": "Participant_ID"})

# ---- follow_up: max days_to_follow_up por paciente (chunked) ----
usecols_fu = ["cases.submitter_id", "follow_ups.days_to_follow_up"]
max_fu = {}
for chunk in pd.read_csv(followup_path, sep="\t", usecols=usecols_fu, chunksize=200000, low_memory=False):
    sid = chunk["cases.submitter_id"].astype(str)
    days = pd.to_numeric(chunk["follow_ups.days_to_follow_up"], errors="coerce")
    m = days.groupby(sid).max()
    for k, v in m.items():
        if pd.isna(v):
            continue
        prev = max_fu.get(k)
        if prev is None or v > prev:
            max_fu[k] = float(v)

pat_df["follow_ups.days_to_follow_up_max"] = pat_df["Participant_ID"].map(max_fu)

# ---- construir OS (time/event) ----
pat_df["follow_up_time"] = pat_df[["follow_ups.days_to_follow_up_max", "diagnoses.days_to_last_follow_up"]].max(axis=1, skipna=True)

pat_df["event"] = (
    (pat_df["demographic.vital_status"].astype(str).str.lower() == "dead") |
    (pat_df["demographic.days_to_death"].notna())
).astype(int)

pat_df["time"] = np.where(
    pat_df["event"] == 1,
    pat_df["demographic.days_to_death"],
    pat_df["follow_up_time"]
)

# compatibilidade com o resto do pipeline
pat_df["__JOIN_ID__"] = pat_df["Participant_ID"].astype(str)
pat_df["age"] = pat_df["demographic.age_at_index"]

# ---- define os_df final (mantém demographic.gender no nome original) ----
os_df = pat_df.copy()

# ---- auditoria rápida ----
audit = {
    "n_patients": int(os_df.shape[0]),
    "n_dead": int(os_df["event"].sum()),
    "n_alive": int((1 - os_df["event"]).sum()),
    "age_non_null": int(os_df["age"].notna().sum()),
    "gender_non_null": int(os_df["demographic.gender"].notna().sum()),
    "iss_non_null": int(os_df["diagnoses.iss_stage"].notna().sum()) if "diagnoses.iss_stage" in os_df.columns else 0
}

audit_path = os.path.join(LOG_DIR, "clinical_input_audit.txt")
with open(audit_path, "w") as f:
    for k, v in audit.items():
        f.write(f"{k}\t{v}\n")

out_os = os.path.join(RES_DIR, "os_df_patient_level.tsv")
os_df.to_csv(out_os, sep="\t", index=False)

print("[OK] os_df reconstruído (paciente-nível).")
print("[OK] salvo:", out_os)
print("[OK] audit:", audit_path)
print("Colunas-chave presentes?",
      {"age": "age" in os_df.columns,
       "demographic.gender": "demographic.gender" in os_df.columns,
       "time": "time" in os_df.columns,
       "event": "event" in os_df.columns,
       "ISS": "diagnoses.iss_stage" in os_df.columns})
os_df[["Participant_ID","__JOIN_ID__","time","event","age","demographic.gender","diagnoses.iss_stage"]].head()


In [ ]:
print("os_df shape:", os_df.shape)
print("\nGender:")
print(os_df["demographic.gender"].value_counts(dropna=False))

if "diagnoses.iss_stage" in os_df.columns:
    print("\nISS:")
    print(os_df["diagnoses.iss_stage"].value_counts(dropna=False))

print("\nEventos:")
print(os_df["event"].value_counts(dropna=False))


In [ ]:
# =========================================================
# 14) MERGE FEATURES ↔ OS (SEM NA→0) — ROBUSTO A NOMES DE COLUNAS
# =========================================================
import os
import pandas as pd

# --- diretório de resultados (compatível com variações) ---
RES_DIR = globals().get("RES_DIR") or globals().get("RESULTS_DIR") or globals().get("OUT_RESULTS_DIR")
if RES_DIR is None:
    RES_DIR = os.path.join(globals().get("OUT_DIR", os.getcwd()), "results")
os.makedirs(RES_DIR, exist_ok=True)

# --- colunas obrigatórias ---
req = ["Participant_ID", "time", "event"]
missing = [c for c in req if c not in os_df.columns]
if missing:
    raise RuntimeError(f"os_df sem colunas obrigatórias: {missing}")

# --- tenta achar a coluna de idade e sexo/gênero sem forçar nome ---
age_candidates = ["age", "Age", "age_at_index", "cases.demographic.age_at_index"]
sex_candidates = ["demographic.gender", "gender", "sex", "Sex", "demographic.sex", "cases.demographic.gender", "cases.demographic.sex"]

age_col = next((c for c in age_candidates if c in os_df.columns), None)
sex_col = next((c for c in sex_candidates if c in os_df.columns), None)

# __JOIN_ID__ é opcional (depende de como você montou o join)
join_col = "__JOIN_ID__" if "__JOIN_ID__" in os_df.columns else None

base_cols = ["Participant_ID"]
if join_col: base_cols.append(join_col)
base_cols += ["time", "event"]
if age_col: base_cols.append(age_col)
if sex_col: base_cols.append(sex_col)

base = os_df[base_cols].copy()

# padroniza nomes (sem criar valores)
rename_map = {}
if age_col and age_col != "age":
    rename_map[age_col] = "age"
if sex_col and sex_col != "sex":
    rename_map[sex_col] = "sex"
base = base.rename(columns=rename_map)

# --- merges: não depende de __JOIN_ID__ nos dfs de métricas ---
def safe_drop(df, col):
    return df.drop(columns=[col]) if col in df.columns else df

features_df = (
    base
    .merge(safe_drop(bp_metrics_df, "__JOIN_ID__"), on="Participant_ID", how="left")
    .merge(safe_drop(hotspot_metrics_df, "__JOIN_ID__"), on="Participant_ID", how="left")
)

export_df_stats(features_df, "features_df_merged")

merged_path = os.path.join(RES_DIR, "survival_features_merged.tsv")
features_df.to_csv(merged_path, sep="\t", index=False)

print(f"[OK] salvo: {merged_path}")
print("[INFO] colunas base usadas:", base_cols)
features_df.head()


In [ ]:
# =========================================================
# 15A) SEX DIFFERENCES — Mann-Whitney U + FDR (BH) + export
#     (Reproducible for Supplementary Table)
# =========================================================

import os
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import mannwhitneyu

# -------------------------
# Config de saída
# -------------------------
# Usa RES_DIR (do notebook) se existir; senão, cai em /content/results
_RES_DIR = None
try:
    _RES_DIR = Path(RES_DIR)  # noqa: F821  (RES_DIR vem de célula anterior no notebook)
    _RES_DIR.mkdir(parents=True, exist_ok=True)
except Exception:
    _RES_DIR = Path("/content/results")
    _RES_DIR.mkdir(parents=True, exist_ok=True)

# -------------------------
# 1) normaliza sexo/gênero para male/female
# -------------------------
def normalize_sex(x):
    if pd.isna(x):
        return np.nan
    v = str(x).strip().lower()
    male_set   = {"male","m","masc","masculino","homem","man"}
    female_set = {"female","f","fem","feminino","mulher","woman"}
    if v in male_set:
        return "male"
    if v in female_set:
        return "female"
    return np.nan  # ignora outros rótulos/valores

df_sex = features_df.copy()

if "sex" not in df_sex.columns:
    raise RuntimeError("features_df não contém a coluna 'sex'. Verifique o merge/renomeação no pipeline.")

df_sex["sex_norm"] = df_sex["sex"].apply(normalize_sex)

# -------------------------
# 2) métricas a testar (ajuste se quiser incluir mais)
# -------------------------
sex_metrics = [
    "BP_total",
    "BP_unique",
    "HotspotCount_P95",
    "HotspotScore_topK",
    "HotspotEntropy",
]

# mantém só métricas existentes no dataframe
sex_metrics = [m for m in sex_metrics if m in df_sex.columns]
if not sex_metrics:
    raise RuntimeError("Nenhuma das métricas esperadas foi encontrada em features_df.")

# -------------------------
# 3) descritivas (mean±sd e mediana [Q1–Q3]) por sexo
# -------------------------
def _q1(x): return x.quantile(0.25)
def _q3(x): return x.quantile(0.75)

desc_rows = []
for metric in sex_metrics:
    tmp = df_sex[["sex_norm", metric]].dropna().copy()
    tmp[metric] = pd.to_numeric(tmp[metric], errors="coerce")
    tmp = tmp.dropna()

    male = tmp.loc[tmp["sex_norm"] == "male", metric]
    female = tmp.loc[tmp["sex_norm"] == "female", metric]

    desc_rows.append({
        "metric": metric,
        "n_male": int(male.shape[0]),
        "n_female": int(female.shape[0]),
        "mean_male": float(male.mean()) if male.shape[0] else np.nan,
        "sd_male": float(male.std(ddof=1)) if male.shape[0] > 1 else np.nan,
        "mean_female": float(female.mean()) if female.shape[0] else np.nan,
        "sd_female": float(female.std(ddof=1)) if female.shape[0] > 1 else np.nan,
        "median_male": float(male.median()) if male.shape[0] else np.nan,
        "q1_male": float(_q1(male)) if male.shape[0] else np.nan,
        "q3_male": float(_q3(male)) if male.shape[0] else np.nan,
        "median_female": float(female.median()) if female.shape[0] else np.nan,
        "q1_female": float(_q1(female)) if female.shape[0] else np.nan,
        "q3_female": float(_q3(female)) if female.shape[0] else np.nan,
    })

desc_sex = pd.DataFrame(desc_rows)
desc_path = _RES_DIR / "supp_table_sex_descriptives_mean_sd_median_iqr.tsv"
desc_sex.to_csv(desc_path, sep="\t", index=False)
print(f"[OK] Descritivas por sexo (mean±sd e mediana[Q1–Q3]) salvas em: {desc_path}")

# -------------------------
# 4) BH-FDR (Benjamini–Hochberg) sem depender de statsmodels
# -------------------------
def bh_fdr(pvals):
    pvals = np.asarray(pvals, dtype=float)
    n = len(pvals)
    order = np.argsort(pvals)
    ranked = pvals[order]
    q = ranked * n / (np.arange(n) + 1)
    q = np.minimum.accumulate(q[::-1])[::-1]
    q = np.clip(q, 0, 1)
    out = np.empty_like(q)
    out[order] = q
    return out

# -------------------------
# 5) Mann-Whitney U + efeito (rank-biserial)
# -------------------------
rows = []
for metric in sex_metrics:
    tmp = df_sex[["sex_norm", metric]].dropna().copy()
    tmp[metric] = pd.to_numeric(tmp[metric], errors="coerce")
    tmp = tmp.dropna()

    male = tmp.loc[tmp["sex_norm"] == "male", metric].values
    female = tmp.loc[tmp["sex_norm"] == "female", metric].values

    # requer ao menos 5 por grupo (ajuste se quiser)
    if (len(male) < 5) or (len(female) < 5):
        rows.append({
            "metric": metric,
            "n_male": int(len(male)),
            "n_female": int(len(female)),
            "U": np.nan,
            "p_value": np.nan,
            "effect_rank_biserial": np.nan,
            "median_male": np.nan if len(male)==0 else float(np.median(male)),
            "median_female": np.nan if len(female)==0 else float(np.median(female)),
            "note": "insuficiente_n"
        })
        continue

    # mannwhitneyu: U é para o primeiro grupo (male)
    U, p = mannwhitneyu(male, female, alternative="two-sided", method="auto")

    # rank-biserial: [-1, 1]
    # positivo => male tende a ser maior
    n1, n2 = len(male), len(female)
    r_rb = (2.0 * U) / (n1 * n2) - 1.0

    rows.append({
        "metric": metric,
        "n_male": int(n1),
        "n_female": int(n2),
        "median_male": float(np.median(male)),
        "median_female": float(np.median(female)),
        "U": float(U),
        "p_value": float(p),
        "effect_rank_biserial": float(r_rb),
        "note": ""
    })

res_sex = pd.DataFrame(rows)

# FDR apenas nas linhas com p válido
mask = res_sex["p_value"].notna()
res_sex.loc[mask, "q_value_bh"] = bh_fdr(res_sex.loc[mask, "p_value"].values)
res_sex.loc[~mask, "q_value_bh"] = np.nan

# export
mw_path = _RES_DIR / "supp_table_sex_mannwhitney.tsv"
res_sex.sort_values("p_value", na_position="last").to_csv(mw_path, sep="\t", index=False)

print(f"[OK] Mann-Whitney por sexo + BH-FDR salvo em: {mw_path}")

# preview (top 10 por p)
display(res_sex.sort_values("p_value", na_position="last").head(10))

# Opcional: tabela combinada (descritivas + p/q) para facilitar citação
try:
    combined = desc_sex.merge(res_sex[["metric", "p_value", "q_value_bh", "effect_rank_biserial"]], on="metric", how="left")
    comb_path = _RES_DIR / "supp_table_sex_combined.tsv"
    combined.to_csv(comb_path, sep="\t", index=False)
    print(f"[OK] Tabela combinada (descritivas + p/q) salva em: {comb_path}")
except Exception as _e:
    print("[WARN] Could not generate combined table:", _e)


In [ ]:
# =========================================================
# 15) COX (BREAKPOINTS/HOTSPOTS) — ROBUSTO A SEX/GENDER
# =========================================================
import os
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index

# diretório resultados
RES_DIR = globals().get("RES_DIR") or globals().get("RESULTS_DIR") or globals().get("OUT_RESULTS_DIR")
if RES_DIR is None:
    RES_DIR = os.path.join(globals().get("OUT_DIR", os.getcwd()), "results")
os.makedirs(RES_DIR, exist_ok=True)

# validações
for c in ["time","event"]:
    if c not in features_df.columns:
        raise RuntimeError(f"features_df sem coluna obrigatória: {c}")

# escolhe lista de métricas automaticamente (tudo numérico exceto colunas-base)
base_exclude = {"Participant_ID","__JOIN_ID__","time","event","age","sex"}
metrics = [c for c in features_df.columns
           if c not in base_exclude and pd.api.types.is_numeric_dtype(features_df[c])]

if len(metrics) == 0:
    raise RuntimeError("Nenhuma métrica numérica encontrada em features_df para rodar Cox.")

# prepara covariáveis
covars = []
if "age" in features_df.columns:
    covars.append("age")

# sexo opcional: cria dummies se existir
df_sex = None
sex_dummy_cols = []
if "sex" in features_df.columns:
    df_sex = features_df[["sex"]].copy()
    df_sex["sex"] = df_sex["sex"].astype(str)
    dummies = pd.get_dummies(df_sex["sex"], prefix="sex", drop_first=True)
    sex_dummy_cols = dummies.columns.tolist()
    # anexa dummies ao dataframe
    features_for_models = pd.concat([features_df.drop(columns=["sex"]), dummies], axis=1)
else:
    features_for_models = features_df.copy()

covars += sex_dummy_cols

rows = []
cph = CoxPHFitter()

for m in metrics:
    cols = ["time","event", m] + covars
    dfm = features_for_models[cols].copy()

    # SEM imputar: remove NA nas colunas usadas
    dfm = dfm.dropna(subset=[m,"time","event"] + covars)

    # evita modelos fracos
    if len(dfm) < 50 or dfm["event"].sum() < 10:
        continue

    # HR por +1 SD (padronização) — só na métrica
    sd = dfm[m].std()
    if not np.isfinite(sd) or sd == 0:
        continue
    dfm[m] = (dfm[m] - dfm[m].mean()) / sd

    try:
        cph.fit(dfm, duration_col="time", event_col="event")
        s = cph.summary.loc[m]

        # C-index (opcional, no mesmo set)
        risk = cph.predict_partial_hazard(dfm).values.ravel()
        cidx = concordance_index(dfm["time"].values, -risk, dfm["event"].values)

        rows.append({
            "metric": m,
            "N": int(dfm.shape[0]),
            "events": int(dfm["event"].sum()),
            "HR_per_1SD": float(np.exp(s["coef"])),
            "CI95_low": float(np.exp(s["coef lower 95%"])),
            "CI95_high": float(np.exp(s["coef upper 95%"])),
            "p": float(s["p"]),
            "c_index_in_sample": float(cidx)
        })
    except Exception as e:
        rows.append({
            "metric": m,
            "N": int(dfm.shape[0]),
            "events": int(dfm["event"].sum()),
            "HR_per_1SD": np.nan,
            "CI95_low": np.nan,
            "CI95_high": np.nan,
            "p": np.nan,
            "c_index_in_sample": np.nan,
            "error": str(e)[:180]
        })

res = pd.DataFrame(rows)
if res.empty:
    raise RuntimeError("Nenhum modelo Cox foi ajustado (poucos eventos ou métricas inválidas).")

# FDR BH
res = res.sort_values("p", na_position="last").reset_index(drop=True)
pvals = res["p"].values.astype(float)
mask = np.isfinite(pvals)
q = np.full_like(pvals, np.nan, dtype=float)
if mask.sum() > 0:
    pv = pvals[mask]
    order = np.argsort(pv)
    ranked = pv[order]
    mtests = len(ranked)
    bh = ranked * mtests / (np.arange(1, mtests+1))
    bh = np.minimum.accumulate(bh[::-1])[::-1]
    q_tmp = np.empty_like(ranked)
    q_tmp[order] = np.clip(bh, 0, 1)
    q[mask] = q_tmp
res["q_FDR_BH"] = q

out_path = os.path.join(RES_DIR, "cox_results_breakpoints_hotspots.tsv")
res.to_csv(out_path, sep="\t", index=False)
print(f"[OK] Cox salvo: {out_path}")
res.head(10)


In [ ]:
import os, pandas as pd, numpy as np

RES_DIR = globals().get("RES_DIR") or globals().get("RESULTS_DIR") or globals().get("OUT_RESULTS_DIR")
if RES_DIR is None:
    RES_DIR = os.path.join(globals().get("OUT_DIR", os.getcwd()), "results")

cox_path = os.path.join(RES_DIR, "cox_results_breakpoints_hotspots.tsv")
print("[CHECK] procurando:", cox_path, "| existe?", os.path.exists(cox_path))

if os.path.exists(cox_path):
    cox_df = pd.read_csv(cox_path, sep="\t")
    print("[CHECK] cox_df shape:", cox_df.shape)
    print("[CHECK] colunas:", list(cox_df.columns))
    print("[CHECK] p non-null:", cox_df["p"].notna().sum() if "p" in cox_df.columns else "NA")
    print("[CHECK] q non-null:", cox_df["q_FDR_BH"].notna().sum() if "q_FDR_BH" in cox_df.columns else "NA")
    display(cox_df.head(10))
else:
    print("[WARN] Não existe arquivo de Cox. A célula 15 não salvou output.")


In [ ]:

# =========================================================
# 15b) KM PLOTS (High vs Low por mediana) — métricas top
# =========================================================
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.plotting import add_at_risk_counts

sig = []
if "cox_df" in globals() and isinstance(cox_df, pd.DataFrame) and not cox_df.empty:
    if "q_FDR_BH" in cox_df.columns:
        sig = cox_df[cox_df["q_FDR_BH"] <= 0.10]["metric"].tolist()
    if not sig:
        sig = cox_df.head(3)["metric"].tolist()

if not sig:
    print("[INFO] Sem métricas para KM (cox_df vazio).")
else:
    km_dir = os.path.join(RES_DIR, "km_plots")
    os.makedirs(km_dir, exist_ok=True)

    for m in sig:
        dfm = features_df[["time","event", m]].dropna(subset=[m,"time","event"]).copy()
        if len(dfm) < 50:
            print(f"[WARN] {m}: n<50. Pulando KM.")
            continue
        med = dfm[m].median()
        dfm["group"] = np.where(dfm[m] >= med, "High", "Low")

        fig = plt.figure(figsize=(7,5))
        ax = plt.gca()

        kmf_low = KaplanMeierFitter()
        kmf_high = KaplanMeierFitter()

        low = dfm["group"].eq("Low")
        high = dfm["group"].eq("High")

        kmf_low.fit(dfm.loc[low,"time"], event_observed=dfm.loc[low,"event"], label="Low")
        kmf_high.fit(dfm.loc[high,"time"], event_observed=dfm.loc[high,"event"], label="High")
        kmf_low.plot_survival_function(ax=ax)
        kmf_high.plot_survival_function(ax=ax)

        lr = logrank_test(dfm.loc[high,"time"], dfm.loc[low,"time"],
                          event_observed_A=dfm.loc[high,"event"], event_observed_B=dfm.loc[low,"event"])

        ax.set_title(f"KM — {m} (median split)\nlog-rank p={lr.p_value:.3g}")
        ax.set_xlabel("Time (days)")
        ax.set_ylabel("Survival probability")

        add_at_risk_counts(kmf_low, kmf_high, ax=ax)
        outp = os.path.join(km_dir, f"KM_{m}.png")
        plt.tight_layout()
        plt.savefig(outp, dpi=200)
        plt.close(fig)

    print(f"[OK] KM plots → {km_dir}")


## Clustering (optional)


In [ ]:

# =========================================================
# 16) CLUSTERING (OPCIONAL) — determinístico + export
# =========================================================
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

cluster_features = ["BP_total","BP_unique","BP_entropy_chr","BP_gini_chr","HotspotCount_P95","HotspotScore_topK","HotspotEntropy"]

Xdf = features_df[["Participant_ID"] + cluster_features].dropna().copy()
if len(Xdf) < 50:
    print(f"[WARN] n={len(Xdf)} insuficiente para clustering.")
else:
    X = np.log1p(Xdf[cluster_features].values.astype(float))
    Xs = StandardScaler().fit_transform(X)

    best_k, best_s = None, -1
    for k in range(2, 7):
        labels = AgglomerativeClustering(n_clusters=k, linkage="ward").fit_predict(Xs)
        s = silhouette_score(Xs, labels)
        if s > best_s:
            best_s, best_k = s, k

    labels = AgglomerativeClustering(n_clusters=best_k, linkage="ward").fit_predict(Xs)
    clusters_df = Xdf[["Participant_ID"]].copy()
    clusters_df["cluster"] = labels.astype(int)
    clusters_df["__JOIN_ID__"] = clusters_df["Participant_ID"].astype("string")

    clus_path = os.path.join(RES_DIR, "cluster_assignments.tsv")
    clusters_df.to_csv(clus_path, sep="\t", index=False)
    export_df_stats(clusters_df, "clusters_df")

    print(f"[OK] clustering k={best_k}, silhouette={best_s:.3f}")
    clusters_df["cluster"].value_counts().sort_index()


In [ ]:

# =========================================================
# 17) SURVIVAL POR CLUSTER (se clusters_df existir)
# =========================================================
from lifelines.statistics import multivariate_logrank_test

if "clusters_df" not in globals() or clusters_df is None or clusters_df.empty:
    print("[INFO] clusters_df não disponível.")
else:
    dfc = os_df.merge(clusters_df[["Participant_ID","cluster"]], on="Participant_ID", how="inner")
    if dfc["cluster"].nunique() < 2:
        print("[INFO] Apenas 1 cluster após join.")
    else:
        res = multivariate_logrank_test(dfc["time"], dfc["cluster"], dfc["event"])
        outp = os.path.join(RES_DIR, "cluster_survival_logrank.txt")
        with open(outp, "w") as f:
            f.write(f"multivariate_logrank_test p={res.p_value}\n")
            f.write(str(res.summary))
        print(f"[OK] log-rank por cluster → {outp}")
        print(res.summary)


# End — main outputs

- `processed/file_metadata.tsv`
- `processed/cnv_segments.parquet`
- `results/cnv_recurrence_by_cytoband.tsv`
- `results/cnv_recurrence_by_bins_1Mb.tsv`
- `results/cox_results_breakpoints_hotspots.tsv`
- `results/cluster_assignments.tsv` (if executed)
- `results/top*_recurrent_regions_by_bins_1Mb.tsv` (if executed)
- `results/top*_recurrent_regions_cox_table.tsv` (if executed)
- `results/recurrent_cnv_km_plots/` (if executed)
- `results/prediction_dataset_top*_bins1Mb.tsv` (if executed)
- `results/pred_survival_*` and `results/pred_iss_*` (if executed)



## Recurrent CNV module (legacy-compatible): top CNVs + adjusted Cox + KM + ISS stratification

This block provides a **legacy-compatible** analysis set:
- identifies **top recurrent CNVs** (exact coordinates) and exports recurrence tables;
- runs **multivariable Cox** per CNV (adjusted for age and sex);
- generates **Kaplan–Meier** plots for selected CNVs;
- optional: **ISS stratification** when ISS is available in `clinical.tsv`;
- optional: a **circos-style** visualization (only if the library is available; execution does not fail otherwise).

> Note: exact-coordinate recurrence depends on segmentation breakpoints; therefore the pipeline also keeps the more robust recurrence summaries by **cytoband** and by **1Mb bins**.



In [ ]:
# =========================================================
# 18) RECURRENT CNV: selecionar TOP regiões por bins (1Mb)
#     (AGORA filtrando CNVs "plotáveis" p/ KM: ambos grupos >= KM_MIN_GROUP)
# =========================================================
import os
import pandas as pd

PROC_DIR = globals().get("PROC_DIR", os.path.join(BASE_OUT_DIR, "processed"))
RES_DIR  = globals().get("RES_DIR",  os.path.join(BASE_OUT_DIR, "results"))
os.makedirs(RES_DIR, exist_ok=True)

BIN_SIZE = int(globals().get("BIN_SIZE", 1_000_000))
TOP_K = int(globals().get("TOP_K_TOPCNV", 10))
KM_MIN_GROUP = int(globals().get("KM_MIN_GROUP", 20))

freq_path  = os.path.join(RES_DIR, f"cnv_recurrence_by_bins_{BIN_SIZE//1_000_000}Mb.tsv")
top_path   = os.path.join(RES_DIR, f"top{TOP_K}_recurrent_regions_by_bins_{BIN_SIZE//1_000_000}Mb.tsv")
edges_path = os.path.join(PROC_DIR, f"cnv_patient_bin_overlaps_{BIN_SIZE//1_000_000}Mb.tsv")

if not os.path.exists(freq_path):
    raise FileNotFoundError("Falta cnv_recurrence_by_bins_1Mb.tsv. Rode a etapa de recorrência antes.")

freq = pd.read_csv(freq_path, sep="\t")

# N da coorte CNV (mesma base usada em Cox/KM)
if not os.path.exists(edges_path):
    raise FileNotFoundError(f"Falta {edges_path}. Rode a etapa de overlaps (edges) antes.")
_edges = pd.read_csv(edges_path, sep="\t", usecols=["Participant_ID"])
COHORT_N_CNV = _edges["Participant_ID"].astype(str).nunique()

print(f"[INFO] COHORT_N_CNV (edges) = {COHORT_N_CNV}")
print(f"[INFO] KM_MIN_GROUP = {KM_MIN_GROUP} | TOP_K = {TOP_K}")

# Filtra CNVs "plotáveis": garante n1>=KM_MIN_GROUP e n0>=KM_MIN_GROUP
freq_ok = freq[
    (freq["n_patients"] >= KM_MIN_GROUP) &
    (freq["n_patients"] <= (COHORT_N_CNV - KM_MIN_GROUP))
].copy()

if freq_ok.empty:
    print("[WARN] Nenhuma região passou o filtro de grupos (KM_MIN_GROUP).")
    print("[WARN] Vou manter a lista original (sem filtro). Isso pode gerar 0 PNGs no KM.")
    freq_use = freq
else:
    freq_use = freq_ok
    print(f"[OK] Regiões elegíveis p/ KM: {len(freq_ok)} / {len(freq)}")

top = (freq_use
       .sort_values(["n_patients"], ascending=False)
       .head(TOP_K)
       .copy())

top.to_csv(top_path, sep="\t", index=False)

print(f"[OK] Top regiões (filtradas p/ KM) exportadas: {top_path}")
display(top.head(10))


In [ ]:
# =========================================================
# 18b) RECURRENT CNV: selecionar TOP CNVs por SegmentID exato (chr:start-end)
#     (filtra "plotáveis" p/ KM: ambos grupos >= KM_MIN_GROUP)
# =========================================================
import os
import pandas as pd
import numpy as np
import re

RES_DIR  = globals().get("RES_DIR",  os.path.join(BASE_OUT_DIR, "results"))
os.makedirs(RES_DIR, exist_ok=True)

TOP_K = int(globals().get("TOP_K_TOPCNV", 10))
KM_MIN_GROUP = int(globals().get("KM_MIN_GROUP", 20))

freq_path = os.path.join(RES_DIR, "cnv_recurrence_by_segmentid_exact__by_patient.tsv")
top_path  = os.path.join(RES_DIR, f"top{TOP_K}_recurrent_segments_by_segmentid_exact.tsv")

if not os.path.exists(freq_path):
    raise FileNotFoundError("Falta cnv_recurrence_by_segmentid_exact__by_patient.tsv. Rode a etapa 8.5 antes.")

if "df_cnvs_hc" not in globals():
    raise RuntimeError("df_cnvs_hc não está em memória. Rode a etapa 8 antes.")

freq = pd.read_csv(freq_path, sep="\t")

COHORT_N_CNV = int(df_cnvs_hc["Participant_ID"].astype(str).nunique())
print(f"[INFO] COHORT_N_CNV (df_cnvs_hc) = {COHORT_N_CNV}")
print(f"[INFO] KM_MIN_GROUP = {KM_MIN_GROUP} | TOP_K = {TOP_K}")

freq_ok = freq[
    (freq["n_patients"] >= KM_MIN_GROUP) &
    (freq["n_patients"] <= (COHORT_N_CNV - KM_MIN_GROUP))
].copy()

if freq_ok.empty:
    print("[WARN] Nenhum SegmentID passou o filtro de grupos (KM_MIN_GROUP).")
    print("[WARN] Vou manter a lista original (sem filtro). Isso pode gerar 0 PNGs no KM.")
    freq_use = freq
else:
    freq_use = freq_ok
    print(f"[OK] Segmentos elegíveis p/ KM: {len(freq_ok)} / {len(freq)}")

top = (freq_use
       .sort_values(["n_patients"], ascending=False)
       .head(TOP_K)
       .copy())

# parse coordenadas
m = top["SegmentID_exact"].astype(str).str.extract(r"(?P<Chromosome>[^:]+):(?P<Start>\d+)-(?P<End>\d+)")
top["Chromosome"] = m["Chromosome"]
top["Start"] = pd.to_numeric(m["Start"], errors="coerce")
top["End"] = pd.to_numeric(m["End"], errors="coerce")

# nome de variável canônico
top["var"] = (
    "CNVEXACT__" +
    top["CNV_Type_Ajustado"].astype(str).str.replace(" ", "_") +
    "__" +
    top["SegmentID_exact"].astype(str).str.replace(":", "_").str.replace("-", "_")
)

top.to_csv(top_path, sep="\t", index=False)
print(f"[OK] Top SegmentIDs (filtrados p/ KM) exportados: {top_path}")
display(top.head(10))


In [ ]:
# =========================================================
# 19) RECURRENT CNV: COX por presença/ausência das TOP CNVs (bins 1Mb)
#     - Ajuste idade/sexo (se existir)
#     - Sem imputação clínica (apenas 0/1 para presença/ausência de CNV nas features)
# =========================================================
import os
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter

PROC_DIR = globals().get("PROC_DIR", os.path.join(BASE_OUT_DIR, "processed"))
RES_DIR  = globals().get("RES_DIR",  os.path.join(BASE_OUT_DIR, "results"))
os.makedirs(RES_DIR, exist_ok=True)

BIN_SIZE = int(globals().get("BIN_SIZE", 1_000_000))
TOP_K = int(globals().get("TOP_K_TOPCNV", 10))

edges_path = os.path.join(PROC_DIR, f"cnv_patient_bin_overlaps_{BIN_SIZE//1_000_000}Mb.tsv")
top_path   = os.path.join(RES_DIR,  f"top{TOP_K}_recurrent_regions_by_bins_{BIN_SIZE//1_000_000}Mb.tsv")

if not os.path.exists(edges_path) or not os.path.exists(top_path):
    raise FileNotFoundError("Faltam arquivos de entrada (edges/top). Rode a célula 18 antes.")

edges = pd.read_csv(edges_path, sep="\t")
top   = pd.read_csv(top_path, sep="\t")

# valida OS
if "os_df" not in globals() or not isinstance(os_df, pd.DataFrame) or os_df.empty:
    raise RuntimeError("os_df não está em memória. Rode a célula 'OS LOCKDOWN' (13) antes.")

df_surv = os_df.copy()
df_surv["Participant_ID"] = df_surv["Participant_ID"].astype(str)

# restringe à coorte com CNV disponível (não assumir ausência fora da coorte)
cnv_patients = set(edges["Participant_ID"].astype(str).unique())
df_surv = df_surv[df_surv["Participant_ID"].isin(cnv_patients)].copy()

if df_surv.shape[0] < 30:
    raise RuntimeError(f"Coorte OS∩CNV muito pequena: {df_surv.shape[0]} pacientes.")

# identifica colunas de tempo/evento
time_col = pick_col(df_surv, ["time", "OS_time", "OS_days", "os_time", "days_to_death_or_last_followup"])
event_col = pick_col(df_surv, ["event", "OS_event", "os_event", "vital_event"])
if time_col is None or event_col is None:
    raise RuntimeError(f"Não achei colunas de tempo/evento em os_df. Colunas: {list(df_surv.columns)[:50]}")

# covariáveis candidatas (sem imputar)
age_col = pick_col(df_surv, ["age", "Age", "age_at_index", "cases.demographic.age_at_index"])
sex_col = pick_col(df_surv, ["sex", "Sex", "gender", "cases.demographic.gender"])

# cria variáveis binárias CNV__<type>__<BinID> para TOP regiões
top = top.copy()
top["var"] = (
    "CNV__" +
    top["CNV_Type_Ajustado"].astype(str).str.replace(" ", "_") +
    "__" +
    top["BinID"].astype(str).str.replace(":", "_").str.replace("-", "_")
)

keep = top[["BinID","CNV_Type_Ajustado","var","n_patients","Chromosome","BinStart","BinEnd"]].copy()

# mapeia edges → apenas TOP regiões
edges2 = edges.merge(keep[["BinID","CNV_Type_Ajustado","var"]], on=["BinID","CNV_Type_Ajustado"], how="inner")
edges2 = edges2.drop_duplicates(["Participant_ID","var"]).copy()
edges2["val"] = 1

X = (edges2
    .pivot(index="Participant_ID", columns="var", values="val")
    .fillna(0)
    .astype(int)
    .reset_index()
)

# monta base de modelagem (OS + features CNV)
df_model_base = df_surv.merge(X, on="Participant_ID", how="left")

# Preencher AUSÊNCIA de CNV (paciente sem ocorrência no Top-K) com 0
cnv_cols = [c for c in df_model_base.columns if c.startswith("CNV__")]
if cnv_cols:
    df_model_base[cnv_cols] = df_model_base[cnv_cols].fillna(0).astype(int)

# normaliza idade/sexo se existirem (sem preencher missing)
covars = []
rename_map = {}
if age_col and age_col != "age":
    rename_map[age_col] = "age"
if sex_col and sex_col != "sex":
    rename_map[sex_col] = "sex"
if rename_map:
    df_model_base = df_model_base.rename(columns=rename_map)

if "age" in df_model_base.columns:
    covars.append("age")

if "sex" in df_model_base.columns:
    # dummies de sexo sem inventar valores
    df_model_base["sex"] = df_model_base["sex"].astype(str)
    sex_dum = pd.get_dummies(df_model_base["sex"], prefix="sex", drop_first=True)
    df_model_base = pd.concat([df_model_base.drop(columns=["sex"]), sex_dum], axis=1)
    covars.extend(sex_dum.columns.tolist())

# expõe variáveis para KM/estadiamento (células 20/21)
globals()["df_model_base"] = df_model_base
globals()["time_col"] = time_col
globals()["event_col"] = event_col

# ---------------------------------------------------------
# Cox por variável (recurrent CNV)
# ---------------------------------------------------------
cph = CoxPHFitter()
rows = []

for _, r in keep.iterrows():
    var = r["var"]
    cols = [time_col, event_col, var] + covars
    tmp = df_model_base[cols].copy().dropna()  # sem imputação

    n_events = int(tmp[event_col].sum())
    if tmp.shape[0] < 30 or n_events < 10:
        rows.append({
            "Region_BinID": r["BinID"],
            "Chromosome": r["Chromosome"],
            "BinStart": int(r["BinStart"]),
            "BinEnd": int(r["BinEnd"]),
            "CNV_Type": r["CNV_Type_Ajustado"],
            "Patients_with_CNV": int(r["n_patients"]),
            "Cohort_N_used": int(tmp.shape[0]),
            "Events_used": n_events,
            "HR": np.nan, "CI95_low": np.nan, "CI95_high": np.nan, "p": np.nan,
            "note": "insuf_eventos_ou_N"
        })
        continue

    try:
        cph.fit(tmp, duration_col=time_col, event_col=event_col)
        s = cph.summary.loc[var]
        hr = float(np.exp(s["coef"]))
        lo = float(np.exp(s["coef lower 95%"]))
        hi = float(np.exp(s["coef upper 95%"]))
        p  = float(s["p"])
        rows.append({
            "Region_BinID": r["BinID"],
            "Chromosome": r["Chromosome"],
            "BinStart": int(r["BinStart"]),
            "BinEnd": int(r["BinEnd"]),
            "CNV_Type": r["CNV_Type_Ajustado"],
            "Patients_with_CNV": int(r["n_patients"]),
            "Cohort_N_used": int(tmp.shape[0]),
            "Events_used": n_events,
            "HR": hr, "CI95_low": lo, "CI95_high": hi, "p": p
        })
    except Exception as e:
        rows.append({
            "Region_BinID": r["BinID"],
            "Chromosome": r["Chromosome"],
            "BinStart": int(r["BinStart"]),
            "BinEnd": int(r["BinEnd"]),
            "CNV_Type": r["CNV_Type_Ajustado"],
            "Patients_with_CNV": int(r["n_patients"]),
            "Cohort_N_used": int(tmp.shape[0]),
            "Events_used": n_events,
            "HR": np.nan, "CI95_low": np.nan, "CI95_high": np.nan, "p": np.nan,
            "error": str(e)[:200]
        })

cox_tab = pd.DataFrame(rows)

# BH-FDR (somente para p não-NaN)
pvals = cox_tab["p"].values
mask = np.isfinite(pvals)
q = np.full_like(pvals, np.nan, dtype=float)
if mask.sum() > 0:
    p = pvals[mask]
    order = np.argsort(p)
    ranked = p[order]
    m = len(ranked)
    qvals = ranked * m / (np.arange(m) + 1)
    qvals = np.minimum.accumulate(qvals[::-1])[::-1]
    q[mask] = qvals[np.argsort(order)]
cox_tab["q_BH"] = q

def prognosis(hr):
    if not np.isfinite(hr):
        return ""
    return "Adverse (HR>1)" if hr > 1 else "Favorable (HR<1)"

cox_tab["Prognosis_from_HR"] = cox_tab["HR"].apply(prognosis)
cox_tab = cox_tab.sort_values("p", na_position="last")

out_cox = os.path.join(RES_DIR, f"top{TOP_K}_recurrent_regions_cox_table.tsv")
cox_tab.to_csv(out_cox, sep="\t", index=False)

print(f"[OK] Tabela Cox (bins) salva: {out_cox}")
display(cox_tab.head(15))


In [ ]:
# =========================================================
# 19b) RECURRENT CNV: COX por presença/ausência das TOP CNVs (SegmentID exato)
#     - Ajuste idade/sexo (se existir)
#     - Sem imputação clínica (apenas 0/1 para presença/ausência de CNV nas features)
# =========================================================
import os
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter

RES_DIR  = globals().get("RES_DIR",  os.path.join(BASE_OUT_DIR, "results"))
os.makedirs(RES_DIR, exist_ok=True)

TOP_K = int(globals().get("TOP_K_TOPCNV", 10))
top_path = os.path.join(RES_DIR, f"top{TOP_K}_recurrent_segments_by_segmentid_exact.tsv")
if not os.path.exists(top_path):
    raise FileNotFoundError("Falta lista TOP SegmentID. Rode a célula 18b antes.")

if "df_cnvs_hc" not in globals() or not isinstance(df_cnvs_hc, pd.DataFrame) or df_cnvs_hc.empty:
    raise RuntimeError("df_cnvs_hc não está em memória. Rode a etapa 8 antes.")
if "os_df" not in globals() or not isinstance(os_df, pd.DataFrame) or os_df.empty:
    raise RuntimeError("os_df não está em memória. Rode a célula 'OS LOCKDOWN' (13) antes.")

top = pd.read_csv(top_path, sep="\t")

# survival base
df_surv = os_df.copy()
df_surv["Participant_ID"] = df_surv["Participant_ID"].astype(str)

# restringe à coorte com CNV disponível (não assumir ausência fora da coorte)
cnv_patients = set(df_cnvs_hc["Participant_ID"].astype(str).unique())
df_surv = df_surv[df_surv["Participant_ID"].isin(cnv_patients)].copy()

if df_surv.shape[0] < 30:
    raise RuntimeError(f"Coorte OS∩CNV muito pequena: {df_surv.shape[0]} pacientes.")

# tempo/evento
time_col_exact = pick_col(df_surv, ["time", "OS_time", "OS_days", "os_time", "days_to_death_or_last_followup"])
event_col_exact = pick_col(df_surv, ["event", "OS_event", "os_event", "vital_event"])
if time_col_exact is None or event_col_exact is None:
    raise RuntimeError(f"Não achei colunas de tempo/evento em os_df. Colunas: {list(df_surv.columns)[:50]}")

# covariáveis candidatas (sem imputar)
age_col = pick_col(df_surv, ["age", "Age", "age_at_index", "cases.demographic.age_at_index"])
sex_col = pick_col(df_surv, ["sex", "Sex", "gender", "cases.demographic.gender"])

# prepara CNVs com SegmentID exato
df_seg = df_cnvs_hc.copy()
df_seg["Participant_ID"] = df_seg["Participant_ID"].astype(str)
df_seg["Chromosome"] = df_seg["Chromosome"].astype(str)
df_seg["Start"] = pd.to_numeric(df_seg["Start"], errors="coerce").astype("Int64")
df_seg["End"]   = pd.to_numeric(df_seg["End"],   errors="coerce").astype("Int64")
df_seg = df_seg.dropna(subset=["Participant_ID","Chromosome","Start","End","CNV_Type_Ajustado"])

df_seg["SegmentID_exact"] = (
    df_seg["Chromosome"] + ":" +
    df_seg["Start"].astype(str) + "-" +
    df_seg["End"].astype(str)
)

# define var canônica (igual à 18b)
df_seg["var"] = (
    "CNVEXACT__" +
    df_seg["CNV_Type_Ajustado"].astype(str).str.replace(" ", "_") +
    "__" +
    df_seg["SegmentID_exact"].astype(str).str.replace(":", "_").str.replace("-", "_")
)

# keep (TOP segmentos) com var
keep = top[["SegmentID_exact","CNV_Type_Ajustado","var","n_patients","Chromosome","Start","End"]].copy()

# edges exatos (presença/ausência por paciente)
edges_exact = df_seg.merge(
    keep[["SegmentID_exact","CNV_Type_Ajustado"]],
    on=["SegmentID_exact","CNV_Type_Ajustado"],
    how="inner"
)
edges_exact = edges_exact.drop_duplicates(["Participant_ID","var"]).copy()
edges_exact["val"] = 1

X_exact = (edges_exact
           .pivot(index="Participant_ID", columns="var", values="val")
           .fillna(0)
           .astype(int)
           .reset_index())

# base modelagem
df_model_base_exact = df_surv.merge(X_exact, on="Participant_ID", how="left")

cnv_cols = [c for c in df_model_base_exact.columns if c.startswith("CNVEXACT__")]
if cnv_cols:
    df_model_base_exact[cnv_cols] = df_model_base_exact[cnv_cols].fillna(0).astype(int)

# normaliza idade/sexo (sem preencher missing)
covars = []
rename_map = {}
if age_col and age_col != "age":
    rename_map[age_col] = "age"
if sex_col and sex_col != "sex":
    rename_map[sex_col] = "sex"
if rename_map:
    df_model_base_exact = df_model_base_exact.rename(columns=rename_map)

if "age" in df_model_base_exact.columns:
    covars.append("age")

if "sex" in df_model_base_exact.columns:
    df_model_base_exact["sex"] = df_model_base_exact["sex"].astype(str)
    sex_dum = pd.get_dummies(df_model_base_exact["sex"], prefix="sex", drop_first=True)
    df_model_base_exact = pd.concat([df_model_base_exact.drop(columns=["sex"]), sex_dum], axis=1)
    covars.extend(sex_dum.columns.tolist())

# expõe para KM/estadiamento (SegmentID exato)
globals()["df_model_base_exact"] = df_model_base_exact
globals()["time_col_exact"] = time_col_exact
globals()["event_col_exact"] = event_col_exact

# Cox por variável
cph = CoxPHFitter()
rows = []

for _, r in keep.iterrows():
    var = r["var"]
    cols = [time_col_exact, event_col_exact, var] + covars
    tmp = df_model_base_exact[cols].copy().dropna()

    n_events = int(tmp[event_col_exact].sum())
    if tmp.shape[0] < 30 or n_events < 10:
        rows.append({
            "SegmentID_exact": r["SegmentID_exact"],
            "Chromosome": r["Chromosome"],
            "Start": int(r["Start"]) if pd.notna(r["Start"]) else np.nan,
            "End": int(r["End"]) if pd.notna(r["End"]) else np.nan,
            "CNV_Type": r["CNV_Type_Ajustado"],
            "Patients_with_CNV": int(r["n_patients"]),
            "Cohort_N_used": int(tmp.shape[0]),
            "Events_used": n_events,
            "HR": np.nan, "CI95_low": np.nan, "CI95_high": np.nan, "p": np.nan,
            "note": "insuf_eventos_ou_N"
        })
        continue

    try:
        cph.fit(tmp, duration_col=time_col_exact, event_col=event_col_exact)
        s = cph.summary.loc[var]
        hr = float(np.exp(s["coef"]))
        lo = float(np.exp(s["coef lower 95%"]))
        hi = float(np.exp(s["coef upper 95%"]))
        p  = float(s["p"])
        rows.append({
            "SegmentID_exact": r["SegmentID_exact"],
            "Chromosome": r["Chromosome"],
            "Start": int(r["Start"]) if pd.notna(r["Start"]) else np.nan,
            "End": int(r["End"]) if pd.notna(r["End"]) else np.nan,
            "CNV_Type": r["CNV_Type_Ajustado"],
            "Patients_with_CNV": int(r["n_patients"]),
            "Cohort_N_used": int(tmp.shape[0]),
            "Events_used": n_events,
            "HR": hr, "CI95_low": lo, "CI95_high": hi, "p": p
        })
    except Exception as e:
        rows.append({
            "SegmentID_exact": r["SegmentID_exact"],
            "Chromosome": r["Chromosome"],
            "Start": int(r["Start"]) if pd.notna(r["Start"]) else np.nan,
            "End": int(r["End"]) if pd.notna(r["End"]) else np.nan,
            "CNV_Type": r["CNV_Type_Ajustado"],
            "Patients_with_CNV": int(r["n_patients"]),
            "Cohort_N_used": int(tmp.shape[0]),
            "Events_used": n_events,
            "HR": np.nan, "CI95_low": np.nan, "CI95_high": np.nan, "p": np.nan,
            "error": str(e)[:200]
        })

cox_tab = pd.DataFrame(rows)

# BH-FDR
pvals = cox_tab["p"].values
mask = np.isfinite(pvals)
q = np.full_like(pvals, np.nan, dtype=float)
if mask.sum() > 0:
    p = pvals[mask]
    order = np.argsort(p)
    ranked = p[order]
    m = len(ranked)
    qvals = ranked * m / (np.arange(m) + 1)
    qvals = np.minimum.accumulate(qvals[::-1])[::-1]
    q[mask] = qvals[np.argsort(order)]
cox_tab["q_BH"] = q

def prognosis(hr):
    if not np.isfinite(hr):
        return ""
    return "Adverse (HR>1)" if hr > 1 else "Favorable (HR<1)"

cox_tab["Prognosis_from_HR"] = cox_tab["HR"].apply(prognosis)
cox_tab = cox_tab.sort_values("p", na_position="last")

out_cox = os.path.join(RES_DIR, f"top{TOP_K}_recurrent_segments_cox_table.tsv")
cox_tab.to_csv(out_cox, sep="\t", index=False)

print(f"[OK] Tabela Cox (SegmentID exato) salva: {out_cox}")
display(cox_tab.head(15))


In [ ]:
# =========================================================
# 20) RECURRENT CNV: KM plots para TOP CNVs — ROBUSTO (sem crash) + filtro de prevalência
# =========================================================
import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

RES_DIR = globals().get("RES_DIR", os.path.join(BASE_OUT_DIR, "results"))
km_dir = os.path.join(RES_DIR, "recurrent_cnv_km_plots")
os.makedirs(km_dir, exist_ok=True)

TOP_K = int(globals().get("TOP_K_TOPCNV", 10))
N_KM  = int(globals().get("TOP_KM_PLOTS", min(10, TOP_K)))

# critério mínimo de tamanho de grupo (ajuste se quiser)
MIN_GROUP = int(globals().get("KM_MIN_GROUP", 20))

cox_path = os.path.join(RES_DIR, f"top{TOP_K}_recurrent_regions_cox_table.tsv")
if not os.path.exists(cox_path):
    raise FileNotFoundError("Tabela Cox não encontrada. Rode a célula 19 antes.")

cox_tab = pd.read_csv(cox_path, sep="\t")

if "df_model_base" not in globals():
    raise RuntimeError("df_model_base não está em memória. Rode a célula 19 antes (na mesma sessão).")
if "time_col" not in globals() or "event_col" not in globals():
    raise RuntimeError("time_col/event_col não estão em memória. Rode a célula 19 antes (na mesma sessão).")

# Base survival (garante time/event presentes)
df_base = df_model_base.dropna(subset=[time_col, event_col]).copy()

def safe_name(s):
    s = re.sub(r"[^A-Za-z0-9_\-]+", "_", str(s))
    return s[:180]

def make_var(binid, cnv_type):
    return "CNV__" + str(cnv_type).replace(" ", "_") + "__" + str(binid).replace(":","_").replace("-","_")

# adiciona var + n_present/n_absent em cox_tab (para filtrar CNVs viáveis para KM)
counts = []
N_total = df_base.shape[0]

for i, r in cox_tab.iterrows():
    binid = r.get("Region_BinID", r.get("Region(BinID)", None))
    cnv_type = r.get("CNV_Type", None)
    if binid is None or cnv_type is None:
        continue

    var = make_var(binid, cnv_type)
    if var not in df_base.columns:
        continue

    # garante 0/1; se tiver NaN, trata como 0 (ausente) apenas para variável binária de CNV
    v = pd.to_numeric(df_base[var], errors="coerce").fillna(0).astype(int)
    n1 = int((v == 1).sum())
    n0 = int((v == 0).sum())

    counts.append((i, var, n1, n0))

if not counts:
    # salva vazio e sai sem quebrar
    km_tab = pd.DataFrame(columns=["Region_BinID","CNV_Type","n_present","n_absent","logrank_p","plot"])
    out_km = os.path.join(RES_DIR, f"top{TOP_K}_km_summary.tsv")
    km_tab.to_csv(out_km, sep="\t", index=False)
    print("[INFO] Nenhuma variável CNV encontrada em df_model_base para KM.")
    print("[OK] Resumo KM vazio salvo:", out_km)
    km_tab

else:
    counts_df = pd.DataFrame(counts, columns=["row_idx","var","n_present","n_absent"]).set_index("row_idx")
    cox2 = cox_tab.join(counts_df, how="left")

    # filtra CNVs com grupos mínimos
    cox2["min_group"] = cox2[["n_present","n_absent"]].min(axis=1)
    viable = cox2.dropna(subset=["n_present","n_absent"]).copy()
    viable = viable[(viable["n_present"] >= MIN_GROUP) & (viable["n_absent"] >= MIN_GROUP)].copy()

    if viable.empty:
        km_tab = pd.DataFrame(columns=["Region_BinID","CNV_Type","n_present","n_absent","logrank_p","plot"])
        out_km = os.path.join(RES_DIR, f"top{TOP_K}_km_summary.tsv")
        km_tab.to_csv(out_km, sep="\t", index=False)

        print(f"[INFO] Nenhuma CNV passou o filtro de KM (MIN_GROUP={MIN_GROUP}).")
        print("Isso é comum quando a CNV é MUITO rara ou MUITO frequente na coorte.")
        print("[DICA] Você pode: (a) reduzir KM_MIN_GROUP, (b) usar citobanda em vez de bin 1Mb, ou (c) avaliar apenas Gains/Losses.")
        print("[OK] Resumo KM vazio salvo:", out_km)
        km_tab

    else:
        # escolhe melhores por p (Cox) e, em empate, por maior min_group e frequência
        viable["p_rank"] = pd.to_numeric(viable.get("p", pd.Series(np.nan, index=viable.index)), errors="coerce").fillna(1.0)

        # tenta usar Patients_with_CNV se existir
        freq_col = "Patients_with_CNV" if "Patients_with_CNV" in viable.columns else None

        sort_cols = ["p_rank", "min_group"]
        ascending = [True, False]
        if freq_col:
            sort_cols.append(freq_col)
            ascending.append(False)

        chosen = viable.sort_values(sort_cols, ascending=ascending).head(N_KM)

        kmf = KaplanMeierFitter()
        km_results = []

        for _, r in chosen.iterrows():
            binid = r.get("Region_BinID", r.get("Region(BinID)", None))
            cnv_type = r.get("CNV_Type", None)
            if binid is None or cnv_type is None:
                continue

            var = make_var(binid, cnv_type)
            if var not in df_base.columns:
                print(f"[WARN] Variável CNV não encontrada para KM: {var}")
                continue

            dfp = df_base[[time_col, event_col, var]].dropna().copy()
            dfp[var] = pd.to_numeric(dfp[var], errors="coerce").fillna(0).astype(int)

            g1 = dfp[dfp[var] == 1]
            g0 = dfp[dfp[var] == 0]

            # guarda tamanhos reais
            n1, n0 = int(g1.shape[0]), int(g0.shape[0])

            if n1 < MIN_GROUP or n0 < MIN_GROUP:
                print(f"[WARN] Grupo pequeno para KM: {binid} {cnv_type} | n1={n1} n0={n0}")
                continue

            lr = logrank_test(g1[time_col], g0[time_col], event_observed_A=g1[event_col], event_observed_B=g0[event_col])
            p_lr = float(lr.p_value)

            plt.figure()
            kmf.fit(g0[time_col], event_observed=g0[event_col], label="CNV absent")
            ax = kmf.plot()
            kmf.fit(g1[time_col], event_observed=g1[event_col], label="CNV present")
            kmf.plot(ax=ax)
            ax.set_title(f"{binid} | {cnv_type} | logrank p={p_lr:.3g}")
            ax.set_xlabel(time_col)
            ax.set_ylabel("Survival probability")

            fig_path = os.path.join(km_dir, f"KM__{safe_name(binid)}__{safe_name(cnv_type)}.png")
            plt.tight_layout()
            plt.savefig(fig_path, dpi=200)
            plt.close()

            km_results.append({
                "Region_BinID": binid,
                "CNV_Type": cnv_type,
                "n_present": n1,
                "n_absent": n0,
                "logrank_p": p_lr,
                "plot": os.path.basename(fig_path)
            })

        # agora: se nada gerou KM, não quebra
        if len(km_results) == 0:
            km_tab = pd.DataFrame(columns=["Region_BinID","CNV_Type","n_present","n_absent","logrank_p","plot"])
            out_km = os.path.join(RES_DIR, f"top{TOP_K}_km_summary.tsv")
            km_tab.to_csv(out_km, sep="\t", index=False)
            print(f"[INFO] Após filtros, nenhuma CNV gerou KM (MIN_GROUP={MIN_GROUP}).")
            print("[OK] Resumo KM vazio salvo:", out_km)
            km_tab
        else:
            km_tab = pd.DataFrame(km_results).sort_values("logrank_p", na_position="last")
            out_km = os.path.join(RES_DIR, f"top{TOP_K}_km_summary.tsv")
            km_tab.to_csv(out_km, sep="\t", index=False)

            print(f"[OK] KM plots em: {km_dir}")
            print(f"[OK] Resumo KM: {out_km}")
            km_tab.head(10)



In [ ]:
# =========================================================
# 20b) RECURRENT CNV: KM plots para TOP CNVs (SegmentID exato) — ROBUSTO + filtro de prevalência
# =========================================================
import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

RES_DIR = globals().get("RES_DIR", os.path.join(BASE_OUT_DIR, "results"))
km_dir = os.path.join(RES_DIR, "recurrent_cnv_km_plots_segment_exact")
os.makedirs(km_dir, exist_ok=True)

TOP_K = int(globals().get("TOP_K_TOPCNV", 10))
N_KM  = int(globals().get("TOP_KM_PLOTS", min(10, TOP_K)))
MIN_GROUP = int(globals().get("KM_MIN_GROUP", 20))

cox_path = os.path.join(RES_DIR, f"top{TOP_K}_recurrent_segments_cox_table.tsv")
if not os.path.exists(cox_path):
    raise FileNotFoundError("Tabela Cox (SegmentID exato) não encontrada. Rode a célula 19b antes.")

cox_tab = pd.read_csv(cox_path, sep="\t")

if "df_model_base_exact" not in globals():
    raise RuntimeError("df_model_base_exact não está em memória. Rode a célula 19b antes (na mesma sessão).")
if "time_col_exact" not in globals() or "event_col_exact" not in globals():
    raise RuntimeError("time_col_exact/event_col_exact não estão em memória. Rode a célula 19b antes (na mesma sessão).")

df_base = df_model_base_exact.dropna(subset=[time_col_exact, event_col_exact]).copy()

def safe_name(s):
    s = re.sub(r"[^A-Za-z0-9_\-]+", "_", str(s))
    return s[:180]

def make_var_exact(segid, cnv_type):
    return "CNVEXACT__" + str(cnv_type).replace(" ", "_") + "__" + str(segid).replace(":","_").replace("-","_")

# adiciona contagens (para filtrar CNVs viáveis para KM)
counts = []
N_total = df_base.shape[0]

for i, r in cox_tab.iterrows():
    segid = r.get("SegmentID_exact", None)
    cnv_type = r.get("CNV_Type", None)
    if segid is None or cnv_type is None:
        continue

    var = make_var_exact(segid, cnv_type)
    if var not in df_base.columns:
        continue

    v = pd.to_numeric(df_base[var], errors="coerce").fillna(0).astype(int)
    n1 = int((v == 1).sum())
    n0 = int((v == 0).sum())
    counts.append((i, var, n1, n0))

if not counts:
    km_tab = pd.DataFrame(columns=["SegmentID_exact","CNV_Type","n_present","n_absent","logrank_p","plot"])
    out_km = os.path.join(RES_DIR, f"top{TOP_K}_km_summary_segment_exact.tsv")
    km_tab.to_csv(out_km, sep="\t", index=False)
    print("[INFO] Nenhuma variável CNVEXACT encontrada em df_model_base_exact para KM.")
    print("[OK] Resumo KM vazio salvo:", out_km)
    km_tab
else:
    counts_df = pd.DataFrame(counts, columns=["row_idx","var","n_present","n_absent"]).set_index("row_idx")
    cox2 = cox_tab.join(counts_df, how="left")

    cox2["min_group"] = cox2[["n_present","n_absent"]].min(axis=1)
    viable = cox2.dropna(subset=["n_present","n_absent"]).copy()
    viable = viable[(viable["n_present"] >= MIN_GROUP) & (viable["n_absent"] >= MIN_GROUP)].copy()

    if viable.empty:
        km_tab = pd.DataFrame(columns=["SegmentID_exact","CNV_Type","n_present","n_absent","logrank_p","plot"])
        out_km = os.path.join(RES_DIR, f"top{TOP_K}_km_summary_segment_exact.tsv")
        km_tab.to_csv(out_km, sep="\t", index=False)

        print(f"[INFO] Nenhuma CNVEXACT passou o filtro de KM (MIN_GROUP={MIN_GROUP}).")
        print("[OK] Resumo KM vazio salvo:", out_km)
        km_tab
    else:
        viable["p_rank"] = pd.to_numeric(viable.get("p", pd.Series(np.nan, index=viable.index)), errors="coerce").fillna(1.0)

        sort_cols = ["p_rank", "min_group"]
        ascending = [True, False]
        if "Patients_with_CNV" in viable.columns:
            sort_cols.append("Patients_with_CNV")
            ascending.append(False)

        chosen = viable.sort_values(sort_cols, ascending=ascending).head(N_KM)

        kmf = KaplanMeierFitter()
        km_results = []

        for _, r in chosen.iterrows():
            segid = r.get("SegmentID_exact", None)
            cnv_type = r.get("CNV_Type", None)
            if segid is None or cnv_type is None:
                continue

            var = make_var_exact(segid, cnv_type)
            if var not in df_base.columns:
                print(f"[WARN] Variável CNVEXACT não encontrada para KM: {var}")
                continue

            dfp = df_base[[time_col_exact, event_col_exact, var]].dropna().copy()
            dfp[var] = pd.to_numeric(dfp[var], errors="coerce").fillna(0).astype(int)

            g1 = dfp[dfp[var] == 1]
            g0 = dfp[dfp[var] == 0]

            n1, n0 = int(g1.shape[0]), int(g0.shape[0])
            if n1 < MIN_GROUP or n0 < MIN_GROUP:
                print(f"[WARN] Grupo pequeno para KM: {segid} {cnv_type} | n1={n1} n0={n0}")
                continue

            lr = logrank_test(g1[time_col_exact], g0[time_col_exact], event_observed_A=g1[event_col_exact], event_observed_B=g0[event_col_exact])
            p_lr = float(lr.p_value)

            plt.figure()
            kmf.fit(g0[time_col_exact], event_observed=g0[event_col_exact], label="CNV absent")
            ax = kmf.plot()
            kmf.fit(g1[time_col_exact], event_observed=g1[event_col_exact], label="CNV present")
            kmf.plot(ax=ax)
            ax.set_title(f"{segid} | {cnv_type} | logrank p={p_lr:.3g}")
            ax.set_xlabel(time_col_exact)
            ax.set_ylabel("Survival probability")

            fig_path = os.path.join(km_dir, f"KM__{safe_name(segid)}__{safe_name(cnv_type)}.png")
            plt.tight_layout()
            plt.savefig(fig_path, dpi=200)
            plt.close()

            km_results.append({
                "SegmentID_exact": segid,
                "CNV_Type": cnv_type,
                "n_present": n1,
                "n_absent": n0,
                "logrank_p": p_lr,
                "plot": os.path.basename(fig_path)
            })

        if len(km_results) == 0:
            km_tab = pd.DataFrame(columns=["SegmentID_exact","CNV_Type","n_present","n_absent","logrank_p","plot"])
            out_km = os.path.join(RES_DIR, f"top{TOP_K}_km_summary_segment_exact.tsv")
            km_tab.to_csv(out_km, sep="\t", index=False)
            print(f"[INFO] Após filtros, nenhuma CNVEXACT gerou KM (MIN_GROUP={MIN_GROUP}).")
            print("[OK] Resumo KM vazio salvo:", out_km)
            km_tab
        else:
            km_tab = pd.DataFrame(km_results).sort_values("logrank_p", na_position="last")
            out_km = os.path.join(RES_DIR, f"top{TOP_K}_km_summary_segment_exact.tsv")
            km_tab.to_csv(out_km, sep="\t", index=False)

            print(f"[OK] KM plots (SegmentID exato) em: {km_dir}")
            print(f"[OK] Resumo KM: {out_km}")
            km_tab.head(10)


In [ ]:
# =========================================================
# 21) RECURRENT CNV: Estratificação por STAGING (TODAS as colunas disponíveis)
#    - Cox por estágio (I/II/III etc.) para as TOP regiões (sem forçar CNVs do banner)
#    - Associação CNV↔estágio (qui-quadrado) para TOP regiões (opcional, se SciPy disponível)
# =========================================================
import os
import re
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter

RES_DIR = globals().get("RES_DIR", os.path.join(BASE_OUT_DIR, "results"))
TOP_K = int(globals().get("TOP_K_TOPCNV", 10))
BIN_SIZE = int(globals().get("BIN_SIZE", 1_000_000))
MIN_STAGE_N = int(globals().get("MIN_STAGE_N", 50))      # mínimo de amostras com estágio válido
MAX_LEVELS = int(globals().get("MAX_STAGE_LEVELS", 10))  # evita variáveis "bagunçadas"
MIN_EVENTS = int(globals().get("MIN_EVENTS_STAGE", 8))   # mínimo de eventos por estágio para Cox

BAD_LEVELS = set([
    "", "nan", "none", "null",
    "unknown", "not reported", "not applicable", "na", "n/a",
    "unspecified", "missing", "not available"
])


cox_path = os.path.join(RES_DIR, f"top{TOP_K}_recurrent_regions_cox_table.tsv")
if not os.path.exists(cox_path):
    raise FileNotFoundError("Tabela Cox não encontrada. Rode a célula 19 antes.")

if "df_model_base" not in globals() or "time_col" not in globals() or "event_col" not in globals():
    raise RuntimeError("Variáveis de modelo não estão em memória. Rode a célula 19 antes (na mesma sessão).")

df_base = df_model_base.copy()

# ---------------------------------------------------------
# Detecta colunas de estadiamento (TODAS)
# Prioridade:
# 1) STAGE_COLUMNS_AVAILABLE (do clinical, anexadas no os_df)
# 2) qualquer coluna do df_base com 'iss_stage' ou termina em 'stage'
# ---------------------------------------------------------
stage_candidates = []
if "STAGE_COLUMNS_AVAILABLE" in globals():
    stage_candidates.extend([c for c in STAGE_COLUMNS_AVAILABLE if c in df_base.columns])

stage_candidates.extend([c for c in df_base.columns if re.search(r"(iss_stage|r_iss|riss|revised.*iss)", str(c), re.I)])
stage_candidates.extend([c for c in df_base.columns if re.search(r"(?:^|\.|_)(stage|staging)(?:$|_|\.)", str(c), re.I)])

# remove duplicatas, mantém ordem
stage_candidates = list(dict.fromkeys(stage_candidates))

# filtra colunas com dados suficientes e níveis razoáveis
usable = []
stage_profile_rows = []

for c in stage_candidates:
    s = df_base[c]
    nn = int(s.notna().sum())
    if nn < MIN_STAGE_N:
        continue
    # normaliza como string pra medir níveis
    lv = s.dropna().astype(str).str.strip()
    lv_low = lv.str.lower()
    lv = lv[~lv_low.isin(BAD_LEVELS)]
    nlev = int(lv.nunique())
    if nlev < 2 or nlev > MAX_LEVELS:
        continue
    usable.append(c)
    stage_profile_rows.append({"stage_col": c, "n_nonnull": nn, "n_levels": nlev, "levels": ", ".join(sorted(lv.unique().tolist())[:MAX_LEVELS])})

stage_profile = pd.DataFrame(stage_profile_rows).sort_values(["n_nonnull","n_levels"], ascending=[False, True])
profile_path = os.path.join(RES_DIR, f"recurrent_cnv_stage_columns_profile.tsv")
stage_profile.to_csv(profile_path, sep="\t", index=False)
print(f"[OK] Perfil de estadiamento salvo: {profile_path}")

if not usable:
    print("[WARN] Nenhuma coluna de estadiamento com dados suficientes (>=MIN_STAGE_N e níveis <=MAX_LEVELS). Pulando.")
else:
    print(f"[OK] Colunas de estadiamento usadas: {usable}")

    # lista de regiões / tipos (para reconstruir varname)
    cox_tab = pd.read_csv(cox_path, sep="\t")
    cox_tab = cox_tab.dropna(subset=["Region_BinID","CNV_Type"]).copy()

    covars = [c for c in df_base.columns if c.startswith("sex_")] + (["age"] if "age" in df_base.columns else [])
    cph = CoxPHFitter()

    # tenta SciPy para qui-quadrado
    try:
        from scipy.stats import chi2_contingency
        HAVE_SCIPY = True
    except Exception:
        HAVE_SCIPY = False

    for stage_col in usable:
        # prepara coluna
        df_stage_base = df_base.copy()
        df_stage_base["_stage_"] = df_stage_base[stage_col].astype(str).str.strip()
        df_stage_base.loc[df_stage_base[stage_col].isna(), "_stage_"] = np.nan
        df_stage_base.loc[df_stage_base["_stage_"].astype(str).str.lower().isin(BAD_LEVELS), "_stage_"] = np.nan

        # distribuição
        dist = (df_stage_base["_stage_"].value_counts(dropna=False)
                .rename_axis("level")
                .reset_index(name="n"))
        dist_path = os.path.join(RES_DIR, f"recurrent_cnv_{re.sub(r'[^A-Za-z0-9]+','_',stage_col)}__distribution.tsv")
        dist.to_csv(dist_path, sep="\t", index=False)

        rows = []
        chi_rows = []

        # Cox por nível
        levels = sorted(df_stage_base["_stage_"].dropna().unique().tolist())
        for level in levels:
            df_s = df_stage_base[df_stage_base["_stage_"] == level].copy()
            if df_s.shape[0] < 30 or int(df_s[event_col].sum()) < MIN_EVENTS:
                continue

            for _, r in cox_tab.iterrows():
                binid = r["Region_BinID"]
                cnv_type = r["CNV_Type"]
                var = "CNV__" + str(cnv_type).replace(" ", "_") + "__" + str(binid).replace(":","_").replace("-","_")
                if var not in df_s.columns:
                    continue

                cols = [time_col, event_col, var] + covars
                tmp = df_s[cols].dropna()
                if tmp.shape[0] < 30 or int(tmp[event_col].sum()) < MIN_EVENTS:
                    continue

                try:
                    cph.fit(tmp, duration_col=time_col, event_col=event_col)
                    s = cph.summary.loc[var]
                    rows.append({
                        "stage_col": stage_col,
                        "stage_level": level,
                        "Region_BinID": binid,
                        "CNV_Type": cnv_type,
                        "N_used": int(tmp.shape[0]),
                        "Events_used": int(tmp[event_col].sum()),
                        "HR": float(np.exp(s["coef"])),
                        "CI95_low": float(np.exp(s["coef lower 95%"])),
                        "CI95_high": float(np.exp(s["coef upper 95%"])),
                        "p": float(s["p"]),
                    })
                except Exception:
                    pass

            # Qui-quadrado (CNV presence vs nível) – para TOP regiões apenas
            if HAVE_SCIPY:
                for _, r in cox_tab.iterrows():
                    binid = r["Region_BinID"]
                    cnv_type = r["CNV_Type"]
                    var = "CNV__" + str(cnv_type).replace(" ", "_") + "__" + str(binid).replace(":","_").replace("-","_")
                    if var not in df_stage_base.columns:
                        continue
                    # tabela 2xK (presença vs nível) calculada no dataset inteiro do stage_col
                    tmp2 = df_stage_base[[var, "_stage_"]].dropna()
                    if tmp2["_stage_"].nunique() < 2:
                        continue
                    ct = pd.crosstab(tmp2["_stage_"], tmp2[var])
                    # garante colunas 0 e 1
                    if 0 not in ct.columns:
                        ct[0] = 0
                    if 1 not in ct.columns:
                        ct[1] = 0
                    ct = ct[[0,1]]
                    if ct.values.sum() < 50:
                        continue
                    try:
                        chi2, pval, dof, exp = chi2_contingency(ct.values)
                        chi_rows.append({
                            "stage_col": stage_col,
                            "Region_BinID": binid,
                            "CNV_Type": cnv_type,
                            "chi2": float(chi2),
                            "dof": int(dof),
                            "p": float(pval),
                            "n_used": int(ct.values.sum())
                        })
                    except Exception:
                        pass

        # salva outputs por stage_col
        slug = re.sub(r"[^A-Za-z0-9]+", "_", stage_col).strip("_")
        if rows:
            stage_tab = pd.DataFrame(rows).sort_values(["stage_level","p"], na_position="last")
            out_stage = os.path.join(RES_DIR, f"top{TOP_K}_cox_by_{slug}.tsv")
            stage_tab.to_csv(out_stage, sep="\t", index=False)
            print(f"[OK] Cox estratificado ({stage_col}) salvo: {out_stage}")
        else:
            print(f"[WARN] Sem modelos Cox suficientes para {stage_col} (N/eventos baixos).")

        if HAVE_SCIPY and chi_rows:
            chi_tab = pd.DataFrame(chi_rows).sort_values("p", na_position="last")
            out_chi = os.path.join(RES_DIR, f"top{TOP_K}_chi2_by_{slug}.tsv")
            chi_tab.to_csv(out_chi, sep="\t", index=False)
            print(f"[OK] Qui-quadrado CNV↔estágio ({stage_col}) salvo: {out_chi}")
        elif not HAVE_SCIPY:
            print("[INFO] SciPy não disponível; pulando qui-quadrado (ok).")


In [ ]:
# =========================================================
# 21b) RECURRENT CNV: Estratificação por STAGING (SegmentID exato)
#    - Cox por estágio (I/II/III etc.) para TOP SegmentIDs
#    - Associação CNV↔estágio (qui-quadrado) para TOP SegmentIDs (opcional, se SciPy disponível)
# =========================================================
import os
import re
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter

RES_DIR = globals().get("RES_DIR", os.path.join(BASE_OUT_DIR, "results"))
TOP_K = int(globals().get("TOP_K_TOPCNV", 10))
MIN_STAGE_N = int(globals().get("MIN_STAGE_N", 50))
MAX_LEVELS = int(globals().get("MAX_STAGE_LEVELS", 10))
MIN_EVENTS = int(globals().get("MIN_EVENTS_STAGE", 8))

BAD_LEVELS = set([
    "", "nan", "none", "null",
    "unknown", "not reported", "not applicable", "na", "n/a",
    "unspecified", "missing", "not available"
])

cox_path = os.path.join(RES_DIR, f"top{TOP_K}_recurrent_segments_cox_table.tsv")
if not os.path.exists(cox_path):
    raise FileNotFoundError("Tabela Cox (SegmentID exato) não encontrada. Rode a célula 19b antes.")

if "df_model_base_exact" not in globals() or "time_col_exact" not in globals() or "event_col_exact" not in globals():
    raise RuntimeError("Variáveis de modelo exato não estão em memória. Rode a célula 19b antes (na mesma sessão).")

df_base = df_model_base_exact.copy()

# Detecta colunas de estadiamento
stage_candidates = []
if "STAGE_COLUMNS_AVAILABLE" in globals():
    stage_candidates.extend([c for c in STAGE_COLUMNS_AVAILABLE if c in df_base.columns])

stage_candidates.extend([c for c in df_base.columns if re.search(r"(iss_stage|r_iss|riss|revised.*iss)", str(c), re.I)])
stage_candidates.extend([c for c in df_base.columns if re.search(r"(?:^|\.|_)(stage|staging)(?:$|_|\.)", str(c), re.I)])

stage_candidates = list(dict.fromkeys(stage_candidates))

usable = []
stage_profile_rows = []

for c in stage_candidates:
    s = df_base[c]
    nn = int(s.notna().sum())
    if nn < MIN_STAGE_N:
        continue
    lv = s.dropna().astype(str).str.strip()
    lv_low = lv.str.lower()
    lv = lv[~lv_low.isin(BAD_LEVELS)]
    nlev = int(lv.nunique())
    if nlev < 2 or nlev > MAX_LEVELS:
        continue
    usable.append(c)
    stage_profile_rows.append({"stage_col": c, "n_nonnull": nn, "n_levels": nlev, "levels": ", ".join(sorted(lv.unique().tolist())[:MAX_LEVELS])})

stage_profile = pd.DataFrame(stage_profile_rows).sort_values(["n_nonnull","n_levels"], ascending=[False, True])
profile_path = os.path.join(RES_DIR, f"recurrent_cnv_stage_columns_profile__segment_exact.tsv")
stage_profile.to_csv(profile_path, sep="\t", index=False)
print(f"[OK] Perfil de estadiamento (SegmentID exato) salvo: {profile_path}")

if not usable:
    print("[WARN] Nenhuma coluna de estadiamento com dados suficientes (>=MIN_STAGE_N e níveis <=MAX_LEVELS). Pulando.")
else:
    print(f"[OK] Colunas de estadiamento usadas: {usable}")

    cox_tab = pd.read_csv(cox_path, sep="\t")
    cox_tab = cox_tab.dropna(subset=["SegmentID_exact","CNV_Type"]).copy()

    covars = [c for c in df_base.columns if c.startswith("sex_")] + (["age"] if "age" in df_base.columns else [])
    cph = CoxPHFitter()

    try:
        from scipy.stats import chi2_contingency
        HAVE_SCIPY = True
    except Exception:
        HAVE_SCIPY = False

    def make_var_exact(segid, cnv_type):
        return "CNVEXACT__" + str(cnv_type).replace(" ", "_") + "__" + str(segid).replace(":","_").replace("-","_")

    for stage_col in usable:
        df_stage_base = df_base.copy()
        df_stage_base["_stage_"] = df_stage_base[stage_col].astype(str).str.strip()
        df_stage_base.loc[df_stage_base[stage_col].isna(), "_stage_"] = np.nan
        df_stage_base.loc[df_stage_base["_stage_"].astype(str).str.lower().isin(BAD_LEVELS), "_stage_"] = np.nan

        dist = (df_stage_base["_stage_"].value_counts(dropna=False)
                .rename_axis("level")
                .reset_index(name="n"))
        dist_path = os.path.join(RES_DIR, f"recurrent_cnv_{re.sub(r'[^A-Za-z0-9]+','_',stage_col)}__distribution__segment_exact.tsv")
        dist.to_csv(dist_path, sep="\t", index=False)

        rows = []
        chi_rows = []

        levels = sorted(df_stage_base["_stage_"].dropna().unique().tolist())
        for level in levels:
            df_s = df_stage_base[df_stage_base["_stage_"] == level].copy()
            if df_s.shape[0] < 30 or int(df_s[event_col_exact].sum()) < MIN_EVENTS:
                continue

            for _, r in cox_tab.iterrows():
                segid = r["SegmentID_exact"]
                cnv_type = r["CNV_Type"]
                var = make_var_exact(segid, cnv_type)
                if var not in df_s.columns:
                    continue

                cols = [time_col_exact, event_col_exact, var] + covars
                tmp = df_s[cols].dropna()
                if tmp.shape[0] < 30 or int(tmp[event_col_exact].sum()) < MIN_EVENTS:
                    continue

                try:
                    cph.fit(tmp, duration_col=time_col_exact, event_col=event_col_exact)
                    s = cph.summary.loc[var]
                    rows.append({
                        "stage_col": stage_col,
                        "stage_level": level,
                        "SegmentID_exact": segid,
                        "CNV_Type": cnv_type,
                        "N_used": int(tmp.shape[0]),
                        "Events_used": int(tmp[event_col_exact].sum()),
                        "HR": float(np.exp(s["coef"])),
                        "CI95_low": float(np.exp(s["coef lower 95%"])),
                        "CI95_high": float(np.exp(s["coef upper 95%"])),
                        "p": float(s["p"]),
                    })
                except Exception:
                    pass

            # Qui-quadrado (presença vs nível) – no dataset inteiro do stage_col
            if HAVE_SCIPY:
                for _, r in cox_tab.iterrows():
                    segid = r["SegmentID_exact"]
                    cnv_type = r["CNV_Type"]
                    var = make_var_exact(segid, cnv_type)
                    if var not in df_stage_base.columns:
                        continue
                    tmp2 = df_stage_base[[var, "_stage_"]].dropna()
                    if tmp2["_stage_"].nunique() < 2:
                        continue
                    ct = pd.crosstab(tmp2["_stage_"], tmp2[var])
                    if 0 not in ct.columns:
                        ct[0] = 0
                    if 1 not in ct.columns:
                        ct[1] = 0
                    ct = ct[[0,1]]
                    if ct.values.sum() < 50:
                        continue
                    try:
                        chi2, pval, dof, exp = chi2_contingency(ct.values)
                        chi_rows.append({
                            "stage_col": stage_col,
                            "SegmentID_exact": segid,
                            "CNV_Type": cnv_type,
                            "chi2": float(chi2),
                            "dof": int(dof),
                            "p": float(pval),
                            "n_used": int(ct.values.sum())
                        })
                    except Exception:
                        pass

        slug = re.sub(r"[^A-Za-z0-9]+", "_", stage_col).strip("_")
        if rows:
            stage_tab = pd.DataFrame(rows).sort_values(["stage_level","p"], na_position="last")
            out_stage = os.path.join(RES_DIR, f"top{TOP_K}_cox_by_{slug}__segment_exact.tsv")
            stage_tab.to_csv(out_stage, sep="\t", index=False)
            print(f"[OK] Cox estratificado (SegmentID exato) ({stage_col}) salvo: {out_stage}")
        else:
            print(f"[WARN] Sem modelos Cox suficientes para {stage_col} (N/eventos baixos).")

        if HAVE_SCIPY and chi_rows:
            chi_tab = pd.DataFrame(chi_rows).sort_values("p", na_position="last")
            out_chi = os.path.join(RES_DIR, f"top{TOP_K}_chi2_by_{slug}__segment_exact.tsv")
            chi_tab.to_csv(out_chi, sep="\t", index=False)
            print(f"[OK] Qui-quadrado CNV↔estágio (SegmentID exato) ({stage_col}) salvo: {out_chi}")
        elif not HAVE_SCIPY:
            print("[INFO] SciPy não disponível — pulando qui-quadrado.")


In [ ]:
# =========================================================
# 22) CHECKLIST PUBLICÁVEL (sanity checks) — sem dados inventados
# =========================================================
import os, sys, platform
import pandas as pd

print("[ENV] python:", sys.version.split()[0], "| platform:", platform.platform())
try:
    import numpy as np
    print("[ENV] numpy:", np.__version__)
except: pass
try:
    import pandas as pd
    print("[ENV] pandas:", pd.__version__)
except: pass
try:
    import pyranges as pr
    print("[ENV] pyranges:", pr.__version__)
except: pass
try:
    import lifelines
    print("[ENV] lifelines:", lifelines.__version__)
except: pass

# diretórios
RAW_DIR  = globals().get("RAW_DIR",  os.path.join(BASE_OUT_DIR, "raw"))
PROC_DIR = globals().get("PROC_DIR", os.path.join(BASE_OUT_DIR, "processed"))
RES_DIR  = globals().get("RES_DIR",  os.path.join(BASE_OUT_DIR, "results"))
LOG_DIR  = globals().get("LOG_DIR",  os.path.join(BASE_OUT_DIR, "logs"))

print("[PATHS]", "RAW:", RAW_DIR)
print("[PATHS]", "PROC:", PROC_DIR)
print("[PATHS]", "RES:", RES_DIR)
print("[PATHS]", "LOG:", LOG_DIR)

# 1) dados em memória (se o usuário executou as etapas)
if "df_cnvs_hc" in globals() and isinstance(df_cnvs_hc, pd.DataFrame):
    print("[DATA] df_cnvs_hc:", df_cnvs_hc.shape, "| pacientes:", df_cnvs_hc["Participant_ID"].nunique())
    if "CNV_Type_Ajustado" in df_cnvs_hc.columns:
        print("[DATA] CNV_Type_Ajustado counts:")
        display(df_cnvs_hc["CNV_Type_Ajustado"].value_counts().head(10))
else:
    print("[WARN] df_cnvs_hc não está em memória (rode etapas 6–8).")

if "os_df" in globals() and isinstance(os_df, pd.DataFrame):
    print("[DATA] os_df:", os_df.shape, "| pacientes:", os_df["Participant_ID"].nunique())
else:
    print("[WARN] os_df não está em memória (rode etapa 13).")

# 2) arquivos esperados (publicação/recurrent CNV)
BIN_SIZE = int(globals().get("BIN_SIZE", 1_000_000))
TOP_K = int(globals().get("TOP_K_TOPCNV", 10))

expected = [
    os.path.join(PROC_DIR, "cnv_patient_cytoband_overlaps.tsv"),
    os.path.join(RES_DIR,  "cnv_recurrence_by_cytoband.tsv"),
    os.path.join(PROC_DIR, f"cnv_patient_bin_overlaps_{BIN_SIZE//1_000_000}Mb.tsv"),
    os.path.join(RES_DIR,  f"cnv_recurrence_by_bins_{BIN_SIZE//1_000_000}Mb.tsv"),
    os.path.join(RES_DIR,  f"top{TOP_K}_recurrent_regions_by_bins_{BIN_SIZE//1_000_000}Mb.tsv"),
    os.path.join(RES_DIR,  f"top{TOP_K}_recurrent_regions_cox_table.tsv"),
    os.path.join(RES_DIR,  f"top{TOP_K}_km_summary.tsv"),
]
for p in expected:
    print("[FILE]", ("OK" if os.path.exists(p) else "MISSING"), "→", p)

# 3) garante que não houve imputação silenciosa: procura por colunas de tempo/evento não-numéricas
if "os_df" in globals() and isinstance(os_df, pd.DataFrame):
    tc = pick_col(os_df, ["time","OS_time","OS_days"])
    ec = pick_col(os_df, ["event","OS_event","OS_status"])
    if tc and ec:
        print("[CHECK] time dtype:", os_df[tc].dtype, "| event dtype:", os_df[ec].dtype)


In [ ]:
import os, numpy as np, pandas as pd

# achar RES_DIR automaticamente
RES_DIR = globals().get("RES_DIR") or globals().get("RESULTS_DIR") or globals().get("OUT_RESULTS_DIR")
if RES_DIR is None:
    RES_DIR = os.path.join(globals().get("OUT_DIR", os.getcwd()), "results")
os.makedirs(RES_DIR, exist_ok=True)

merged_path = os.path.join(RES_DIR, "survival_features_merged.tsv")

if "df_surv" in globals() and isinstance(df_surv, pd.DataFrame) and {"time","event"}.issubset(df_surv.columns):
    df_ml = df_surv.copy()
elif os.path.exists(merged_path):
    df_ml = pd.read_csv(merged_path, sep="\t")
else:
    raise FileNotFoundError("Não achei df_surv em memória nem survival_features_merged.tsv em RES_DIR.")

# colunas essenciais
for c in ["Participant_ID","time","event"]:
    if c not in df_ml.columns:
        raise RuntimeError(f"Dataset survival sem coluna obrigatória: {c}")

# remove linhas inválidas
df_ml = df_ml.dropna(subset=["time","event"]).copy()
df_ml = df_ml[df_ml["time"] >= 0].copy()

# define features: tudo que é numérico e NÃO é time/event/ID
drop_cols = {"Participant_ID","time","event"}
num_cols = [c for c in df_ml.columns if c not in drop_cols and pd.api.types.is_numeric_dtype(df_ml[c])]
if len(num_cols) < 3:
    raise RuntimeError(f"Poucas features numéricas detectadas: {num_cols}")

# tira colunas constantes
good = []
for c in num_cols:
    if df_ml[c].nunique(dropna=True) > 1:
        good.append(c)
num_cols = good

# treino/teste estratificado por evento
from sklearn.model_selection import train_test_split
train_idx, test_idx = train_test_split(
    df_ml.index, test_size=0.25, random_state=42,
    stratify=df_ml["event"].astype(int)
)
train_df = df_ml.loc[train_idx].copy()
test_df  = df_ml.loc[test_idx].copy()

print("[OK] Dataset:", df_ml.shape, "| Eventos:", int(df_ml["event"].sum()))
print("[OK] Features numéricas usadas:", len(num_cols))


In [ ]:
# =========================================================
# SUPPL FIG) Pearson Correlation: CNV Burden vs Clinical
# (gera heatmap + matriz TSV em results/)
# =========================================================
import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt

# resolve pasta de resultados sem depender de uma única variável
RES_DIR = globals().get("RES_DIR") or globals().get("RESULTS_DIR") or os.path.join(globals().get("BASE_OUT_DIR", "."), "results")
os.makedirs(RES_DIR, exist_ok=True)

# checagens mínimas
for _v in ["df_cnvs_hc", "os_df"]:
    if _v not in globals():
        raise RuntimeError(f"Variável obrigatória não encontrada em memória: {_v}. Rode as etapas anteriores.")

# --- CNV burden por cromossomo (contagem de segmentos HC) ---
burden = (
    df_cnvs_hc.groupby(["Participant_ID","Chromosome"])
    .size()
    .unstack(fill_value=0)
)

# padroniza nomes chr*
def _chr_name(c):
    c = str(c)
    return "chr" + c.replace("chr","")
burden.columns = [_chr_name(c) for c in burden.columns]

# ordena cromossomos canônicos
ordered = [f"chr{i}" for i in range(1,23)] + ["chrX","chrY","chrM"]
cols = [c for c in ordered if c in burden.columns] + [c for c in burden.columns if c not in ordered]
burden = burden[cols]

burden["Total_CNVs"] = burden.sum(axis=1)

# --- merge com clinical/os_df (somente colunas numéricas úteis) ---
clin = os_df.set_index("Participant_ID").copy()

# indicadores opcionais
if "demographic.gender" in clin.columns:
    g = clin["demographic.gender"].astype(str).str.lower()
    clin["demographic.gender_male"] = g.isin(["male","m"]).astype(int)

# “High burden” (top quartil)
thr = burden["Total_CNVs"].quantile(0.75)
clin["Is_High_CNV_Burden"] = (burden["Total_CNVs"] >= thr).astype(int)

# candidatos clínicos (usa apenas os que existirem)
cand = [
    "age",
    "time","event",
    "demographic.days_to_death",
    "diagnoses.days_to_last_follow_up",
    "follow_ups.days_to_follow_up_max",
    "demographic.gender_male",
    "Is_High_CNV_Burden",
]
cand = [c for c in cand if c in clin.columns]

corr_df = burden.join(clin[cand], how="inner")
corr_df = corr_df.select_dtypes(include=[np.number]).copy()

corr = corr_df.corr(method="pearson")

# --- plot heatmap com anotações ---
# plot (mesmo estilo vermelho/azul do “clássico”)
fig = plt.figure(figsize=(18, 15))
ax = plt.gca()
im = ax.imshow(corr.values, aspect="auto", cmap="coolwarm", vmin=-1, vmax=1)
  # <-- aqui
plt.colorbar(im, fraction=0.046, pad=0.04)

ax.set_xticks(range(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=90, fontsize=7)
ax.set_yticks(range(len(corr.index)))
ax.set_yticklabels(corr.index, fontsize=7)

for i in range(corr.shape[0]):
    for j in range(corr.shape[1]):
        ax.text(j, i, f"{corr.values[i,j]:.2f}", ha="center", va="center", fontsize=6)

ax.set_title("Pearson Correlation Matrix: CNV Burden vs. Clinical Features", fontsize=14)
plt.tight_layout()

out_png = os.path.join(RES_DIR, "pearson_corr_cnv_burden_vs_clinical.png")
plt.savefig(out_png, dpi=200, bbox_inches="tight")
plt.close()

out_tsv = os.path.join(RES_DIR, "pearson_corr_cnv_burden_vs_clinical.tsv")
corr.to_csv(out_tsv, sep="\t")

print("[OK] Heatmap:", out_png)
print("[OK] Matrix TSV:", out_tsv)


In [ ]:
# =========================================================
# MAIN FIG) KM para TOP CNVs (presença/ausência) — estilo banner
# (gera plots + km_logrank_summary.tsv em results/)
# =========================================================
import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

RES_DIR = globals().get("RES_DIR") or globals().get("RESULTS_DIR") or os.path.join(globals().get("BASE_OUT_DIR", "."), "results")
os.makedirs(RES_DIR, exist_ok=True)

TOP_K = int(globals().get("TOP_K_TOPCNV", 10))
KM_MIN_GROUP = int(globals().get("KM_MIN_GROUP", 20))

# precisa ter df_model_base/time_col/event_col gerados no bloco recurrent CNV
for _v in ["df_model_base", "time_col", "event_col"]:
    if _v not in globals():
        raise RuntimeError(f"Variável obrigatória não encontrada em memória: {_v}. Rode as etapas recurrent CNV antes.")

df_base = df_model_base.dropna(subset=[time_col, event_col]).copy()

cox_path = os.path.join(RES_DIR, f"top{TOP_K}_recurrent_regions_cox_table.tsv")
if not os.path.exists(cox_path):
    raise FileNotFoundError(f"Não achei {cox_path}. Rode a célula do COX recurrent CNV antes.")

cox_tab = pd.read_csv(cox_path, sep="\t")

km_dir = os.path.join(RES_DIR, "KM_TOPCNV_PRESENCE")
os.makedirs(km_dir, exist_ok=True)

def safe_name(s):
    s = re.sub(r"[^A-Za-z0-9_\-]+", "_", str(s))
    return s[:180]

def make_var(binid, cnv_type):
    return "CNV__" + str(cnv_type).replace(" ", "_") + "__" + str(binid).replace(":","_").replace("-","_")

km_results = []
kmf = KaplanMeierFitter()

for _, r in cox_tab.head(TOP_K).iterrows():
    binid = r.get("Region_BinID")
    cnv_type = r.get("CNV_Type")
    if pd.isna(binid) or pd.isna(cnv_type):
        continue

    var = make_var(binid, cnv_type)
    if var not in df_base.columns:
        continue

    d = df_base[[time_col, event_col, var]].dropna().copy()
    d[var] = pd.to_numeric(d[var], errors="coerce").fillna(0).astype(int)

    g0 = d[d[var]==0]
    g1 = d[d[var]==1]

    if g0.shape[0] < KM_MIN_GROUP or g1.shape[0] < KM_MIN_GROUP:
        continue

    lr = logrank_test(
        g1[time_col], g0[time_col],
        event_observed_A=g1[event_col],
        event_observed_B=g0[event_col]
    )
    p_lr = float(lr.p_value)

    plt.figure(figsize=(8,6))
    kmf.fit(g0[time_col], event_observed=g0[event_col], label=f"No CNV (N={g0.shape[0]})")
    ax = kmf.plot()
    kmf.fit(g1[time_col], event_observed=g1[event_col], label=f"With CNV (N={g1.shape[0]})")
    kmf.plot(ax=ax)

    ax.set_title(f"Kaplan–Meier survival para {binid} ({cnv_type})")
    ax.set_xlabel("Time (days)")
    ax.set_ylabel("Survival probability")
    ax.text(0.62, 0.74, f"Log-rank p: {p_lr:.4f}", transform=ax.transAxes,
            bbox=dict(facecolor="white", edgecolor="gray"))

    fig_path = os.path.join(km_dir, f"KM__{safe_name(binid)}__{safe_name(cnv_type)}.png")
    plt.tight_layout()
    plt.savefig(fig_path, dpi=200, bbox_inches="tight")
    plt.close()

    km_results.append({
        "Region_BinID": binid,
        "CNV_Type": cnv_type,
        "N_no_CNV": int(g0.shape[0]),
        "N_with_CNV": int(g1.shape[0]),
        "Events_no_CNV": int(pd.to_numeric(g0[event_col], errors="coerce").fillna(0).sum()),
        "Events_with_CNV": int(pd.to_numeric(g1[event_col], errors="coerce").fillna(0).sum()),
        "logrank_p": p_lr,
        "plot": os.path.basename(fig_path)
    })

km_tab = pd.DataFrame(km_results)
out_km = os.path.join(RES_DIR, "km_logrank_summary.tsv")

if km_tab.empty:
    km_tab = pd.DataFrame(columns=["Region_BinID","CNV_Type","N_no_CNV","N_with_CNV","Events_no_CNV","Events_with_CNV","logrank_p","plot"])
    km_tab.to_csv(out_km, sep="\t", index=False)
    print(f"[INFO] Nenhuma CNV em TOP{TOP_K} passou KM_MIN_GROUP={KM_MIN_GROUP}. Resumo vazio salvo:", out_km)
else:
    km_tab = km_tab.sort_values("logrank_p", na_position="last")
    km_tab.to_csv(out_km, sep="\t", index=False)
    print("[OK] KM plots em:", km_dir)
    print("[OK] Resumo KM:", out_km)
    display(km_tab.head(10))


In [ ]:
# =========================================================
# FIND + PREVIEW: KM plots gerados
# =========================================================
import os, glob
from IPython.display import Image, display

RES_DIR = globals().get("RES_DIR", None)
if RES_DIR is None:
    # fallback: tenta achar o results mais recente
    cand = sorted(glob.glob("outputs/run_*/results"), reverse=True)
    if not cand:
        raise RuntimeError("Não achei RES_DIR nem outputs/run_*/results.")
    RES_DIR = cand[0]

km_dir = os.path.join(RES_DIR, "KM_TOPCNV_PRESENCE")
print("[INFO] RES_DIR =", RES_DIR)
print("[INFO] KM dir  =", km_dir)

pngs = sorted(glob.glob(os.path.join(km_dir, "*.png")))
print("[OK] PNGs encontrados:", len(pngs))
for p in pngs[:15]:
    print(" -", os.path.basename(p))

# preview dos primeiros 6
for p in pngs[:6]:
    display(Image(filename=p))


In [ ]:
import os, glob, pandas as pd

RES_DIR = globals().get("RES_DIR", None)
if RES_DIR is None:
    cand = sorted(glob.glob("outputs/run_*/results"), reverse=True)
    RES_DIR = cand[0] if cand else None
print("[INFO] RES_DIR =", RES_DIR)

TOP_K = int(globals().get("TOP_K_TOPCNV", 10))
cox_path = os.path.join(RES_DIR, f"top{TOP_K}_recurrent_regions_cox_table.tsv")
print("[INFO] cox_path =", cox_path, "| exists:", os.path.exists(cox_path))

cox_tab = pd.read_csv(cox_path, sep="\t") if os.path.exists(cox_path) else pd.DataFrame()
print("\n[INFO] Cox columns:", list(cox_tab.columns))
display(cox_tab.head(3))

# mostra como os CNV features existem no df_model_base
cnv_cols = [c for c in df_model_base.columns if str(c).startswith("CNV__")]
print("\n[INFO] CNV feature cols em df_model_base:", len(cnv_cols))
print("Exemplos:", cnv_cols[:10])


In [ ]:
# =========================================================
# 22.9) PREDICTION DATASET: global features (BP/hotspots/age/sex) + locus TOP-K (1Mb bins)
# - Builds df_pred used by the "23) Prediction models" section
# - If the TOP-K file is missing, it is generated from the patient-bin overlap table
# Outputs are saved in RES_DIR
# =========================================================

import os
import re
import numpy as np
import pandas as pd

# --- directories ---
RES_DIR  = globals().get("RES_DIR") or os.path.join(globals().get("OUT_DIR", os.getcwd()), "results")
PROC_DIR = globals().get("PROC_DIR") or os.path.join(globals().get("BASE_OUT_DIR", os.getcwd()), "processed")
os.makedirs(RES_DIR, exist_ok=True)

# --- parameters ---
BIN_SIZE   = int(globals().get("BIN_SIZE", 1_000_000))
BIN_MB     = BIN_SIZE // 1_000_000
TOP_K_PRED = int(globals().get("TOP_K_PRED", 50))  # increase to 100 if needed

# --- base features ---
merged_path = os.path.join(RES_DIR, "survival_features_merged.tsv")
if "features_df" in globals() and isinstance(features_df, pd.DataFrame) and not features_df.empty:
    base = features_df.copy()
elif os.path.exists(merged_path):
    base = pd.read_csv(merged_path, sep="\t")
else:
    raise FileNotFoundError("Missing base features: neither `features_df` in memory nor `survival_features_merged.tsv` in RES_DIR.")

# (optional) bring ISS from os_df (survival_features_merged.tsv may not include it)
if "os_df" in globals() and isinstance(os_df, pd.DataFrame) and "diagnoses.iss_stage" in os_df.columns:
    base = base.merge(os_df[["Participant_ID", "diagnoses.iss_stage"]], on="Participant_ID", how="left")

# --- locus table inputs ---
edges_path = os.path.join(PROC_DIR, f"cnv_patient_bin_overlaps_{BIN_MB}Mb.tsv")
top_path   = os.path.join(RES_DIR,  f"top{TOP_K_PRED}_recurrent_regions_by_bins_{BIN_MB}Mb.tsv")

if not os.path.exists(edges_path):
    raise FileNotFoundError(f"Missing overlap table: {edges_path}. Run the CNV x bin overlap export step first.")

# read edges with safer dtypes (avoid mixed-type warnings)
edges = pd.read_csv(edges_path, sep="\t", low_memory=False)

# detect required columns robustly
cols = {c.lower(): c for c in edges.columns}
participant_col = cols.get("participant_id") or cols.get("participant") or cols.get("patient_id")
bin_col = cols.get("binid") or cols.get("bin_id") or cols.get("bin")
type_col = cols.get("cnv_type_ajustado") or cols.get("cnv_type") or cols.get("type")

missing = [x for x in [participant_col, bin_col, type_col] if x is None]
if missing:
    raise RuntimeError(f"Could not detect required columns in overlap table. Found columns: {list(edges.columns)[:40]}")

# normalize dtypes
edges[participant_col] = edges[participant_col].astype(str)
edges[bin_col] = edges[bin_col].astype(str)
edges[type_col] = edges[type_col].astype(str)

# --- build TOP-K if missing ---
if not os.path.exists(top_path):
    n_total = int(edges[participant_col].nunique())
    freq = (
        edges.groupby([bin_col, type_col])[participant_col]
        .nunique()
        .reset_index(name="n_patients")
    )
    freq["pct_patients"] = (freq["n_patients"] / max(n_total, 1)) * 100.0
    freq = freq.sort_values(["pct_patients", "n_patients"], ascending=False).head(TOP_K_PRED).copy()

    # standardize column names for downstream
    freq = freq.rename(columns={bin_col: "BinID", type_col: "CNV_Type"})
    freq.to_csv(top_path, sep="\t", index=False)
    print("[OK] TOP-K recurrent regions file generated:", top_path)
else:
    freq = pd.read_csv(top_path, sep="\t")
    # accept either legacy or new column naming
    if "BinID" not in freq.columns:
        if bin_col in freq.columns:
            freq = freq.rename(columns={bin_col: "BinID"})
    if "CNV_Type" not in freq.columns:
        if type_col in freq.columns:
            freq = freq.rename(columns={type_col: "CNV_Type"})
    print("[OK] TOP-K recurrent regions file found:", top_path)

# --- make safe variable names: CNV__<TYPE>__<BinID> ---
def _slug(s: str) -> str:
    s = str(s)
    s = s.replace(":", "_").replace("-", "_").replace(" ", "_")
    s = re.sub(r"[^0-9a-zA-Z_]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

freq = freq.copy()
freq["var"] = "CNV__" + freq["CNV_Type"].map(_slug) + "__" + freq["BinID"].map(_slug)

keep = freq[["BinID", "CNV_Type", "var"]].drop_duplicates()

# align edges column names to expected "BinID"/"CNV_Type"
edges2 = edges.rename(columns={bin_col: "BinID", type_col: "CNV_Type", participant_col: "Participant_ID"}).copy()

edges2 = (
    edges2.merge(keep, on=["BinID", "CNV_Type"], how="inner")
         .drop_duplicates(["Participant_ID", "var"])
)
edges2["val"] = 1

X_locus = (
    edges2.pivot(index="Participant_ID", columns="var", values="val")
    .fillna(0).astype(int)
    .reset_index()
)

df_pred = base.merge(X_locus, on="Participant_ID", how="left")
locus_cols = [c for c in df_pred.columns if c.startswith("CNV__")]
df_pred[locus_cols] = df_pred[locus_cols].fillna(0).astype(int)

# --- save dataset for auditability ---
pred_path = os.path.join(RES_DIR, f"prediction_dataset_top{TOP_K_PRED}_bins{BIN_MB}Mb.tsv")
df_pred.to_csv(pred_path, sep="\t", index=False)

print("[OK] df_pred:", df_pred.shape, "| locus cols:", len(locus_cols))
print("[OK] saved:", pred_path)
df_pred.head()


In [ ]:
# =========================================================
# PREDICTION DATASET: global features (BP/hotspots/age/sex) + locus TOP-K (1Mb bins)
# - Builds df_pred for the predictive models section
# - If the TOP-K file is missing, it is generated from the patient-bin overlap table
# Outputs are saved in RES_DIR
# =========================================================

import os
import re
import numpy as np
import pandas as pd

# --- directories ---
RES_DIR  = globals().get("RES_DIR") or os.path.join(globals().get("OUT_DIR", os.getcwd()), "results")
PROC_DIR = globals().get("PROC_DIR") or os.path.join(globals().get("BASE_OUT_DIR", os.getcwd()), "processed")
os.makedirs(RES_DIR, exist_ok=True)

# --- parameters ---
BIN_SIZE   = int(globals().get("BIN_SIZE", 1_000_000))
BIN_MB     = BIN_SIZE // 1_000_000
TOP_K_PRED = int(globals().get("TOP_K_PRED", 50))  # increase to 100 if needed

# --- base features ---
merged_path = os.path.join(RES_DIR, "survival_features_merged.tsv")
if "features_df" in globals() and isinstance(features_df, pd.DataFrame) and not features_df.empty:
    base = features_df.copy()
elif os.path.exists(merged_path):
    base = pd.read_csv(merged_path, sep="\t")
else:
    raise FileNotFoundError("Missing base features: neither `features_df` in memory nor `survival_features_merged.tsv` in RES_DIR.")

# (optional) bring ISS from os_df (survival_features_merged.tsv may not include it)
if "os_df" in globals() and isinstance(os_df, pd.DataFrame) and "diagnoses.iss_stage" in os_df.columns:
    base = base.merge(os_df[["Participant_ID", "diagnoses.iss_stage"]], on="Participant_ID", how="left")

# --- locus table inputs ---
edges_path = os.path.join(PROC_DIR, f"cnv_patient_bin_overlaps_{BIN_MB}Mb.tsv")
top_path   = os.path.join(RES_DIR,  f"top{TOP_K_PRED}_recurrent_regions_by_bins_{BIN_MB}Mb.tsv")

if not os.path.exists(edges_path):
    raise FileNotFoundError(f"Missing overlap table: {edges_path}. Run the CNV x bin overlap export step first.")

# read edges with safer dtypes (avoid mixed-type warnings)
edges = pd.read_csv(edges_path, sep="\t", low_memory=False)

# detect required columns robustly
cols = {c.lower(): c for c in edges.columns}
participant_col = cols.get("participant_id") or cols.get("participant") or cols.get("patient_id")
bin_col = cols.get("binid") or cols.get("bin_id") or cols.get("bin")
type_col = cols.get("cnv_type_ajustado") or cols.get("cnv_type") or cols.get("type")

missing = [x for x in [participant_col, bin_col, type_col] if x is None]
if missing:
    raise RuntimeError(f"Could not detect required columns in overlap table. Found columns: {list(edges.columns)[:40]}")

# normalize dtypes
edges[participant_col] = edges[participant_col].astype(str)
edges[bin_col] = edges[bin_col].astype(str)
edges[type_col] = edges[type_col].astype(str)

# --- build TOP-K if missing ---
if not os.path.exists(top_path):
    n_total = int(edges[participant_col].nunique())
    freq = (
        edges.groupby([bin_col, type_col])[participant_col]
        .nunique()
        .reset_index(name="n_patients")
    )
    freq["pct_patients"] = (freq["n_patients"] / max(n_total, 1)) * 100.0
    freq = freq.sort_values(["pct_patients", "n_patients"], ascending=False).head(TOP_K_PRED).copy()

    # standardize column names for downstream
    freq = freq.rename(columns={bin_col: "BinID", type_col: "CNV_Type"})
    freq.to_csv(top_path, sep="\t", index=False)
    print("[OK] TOP-K recurrent regions file generated:", top_path)
else:
    freq = pd.read_csv(top_path, sep="\t")
    # accept either legacy or new column naming
    if "BinID" not in freq.columns:
        if bin_col in freq.columns:
            freq = freq.rename(columns={bin_col: "BinID"})
    if "CNV_Type" not in freq.columns:
        if type_col in freq.columns:
            freq = freq.rename(columns={type_col: "CNV_Type"})
    print("[OK] TOP-K recurrent regions file found:", top_path)

# --- make safe variable names: CNV__<TYPE>__<BinID> ---
def _slug(s: str) -> str:
    s = str(s)
    s = s.replace(":", "_").replace("-", "_").replace(" ", "_")
    s = re.sub(r"[^0-9a-zA-Z_]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

freq = freq.copy()
freq["var"] = "CNV__" + freq["CNV_Type"].map(_slug) + "__" + freq["BinID"].map(_slug)

keep = freq[["BinID", "CNV_Type", "var"]].drop_duplicates()

# align edges column names to expected "BinID"/"CNV_Type"
edges2 = edges.rename(columns={bin_col: "BinID", type_col: "CNV_Type", participant_col: "Participant_ID"}).copy()

edges2 = (
    edges2.merge(keep, on=["BinID", "CNV_Type"], how="inner")
         .drop_duplicates(["Participant_ID", "var"])
)
edges2["val"] = 1

X_locus = (
    edges2.pivot(index="Participant_ID", columns="var", values="val")
    .fillna(0).astype(int)
    .reset_index()
)

df_pred = base.merge(X_locus, on="Participant_ID", how="left")
locus_cols = [c for c in df_pred.columns if c.startswith("CNV__")]
df_pred[locus_cols] = df_pred[locus_cols].fillna(0).astype(int)

# --- save dataset for auditability ---
pred_path = os.path.join(RES_DIR, f"prediction_dataset_top{TOP_K_PRED}_bins{BIN_MB}Mb.tsv")
df_pred.to_csv(pred_path, sep="\t", index=False)

print("[OK] df_pred:", df_pred.shape, "| locus cols:", len(locus_cols))
print("[OK] saved:", pred_path)
df_pred.head()


In [ ]:
# =========================================================
# 23) PREDICTION MODELS (OPTIONAL / REPORTABLE)  [ANTI-LEAK + STABILITY]
#  - Survival risk prediction (penalized Cox) -> test C-index (no leakage)
#  - ISS stage prediction (multiclass) -> macro-F1 / accuracy
#  Outputs saved in RES_DIR
# =========================================================

import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.linear_model import LogisticRegression

from lifelines import CoxPHFitter
from lifelines.utils import concordance_index

np.random.seed(42)

# ---------------------------------------------------------
# A) Ensure RES_DIR exists
# ---------------------------------------------------------
RES_DIR = globals().get("RES_DIR") or os.path.join(globals().get("OUT_DIR", os.getcwd()), "results")
os.makedirs(RES_DIR, exist_ok=True)

# ---------------------------------------------------------
# 0) Get df_pred (preferred) or df_ml
# ---------------------------------------------------------
if "df_pred" in globals() and isinstance(df_pred, pd.DataFrame) and not df_pred.empty:
    df_model = df_pred.copy()
elif "df_ml" in globals() and isinstance(df_ml, pd.DataFrame) and not df_ml.empty:
    df_model = df_ml.copy()
else:
    raise RuntimeError("Missing `df_pred` and `df_ml`. Run the prediction dataset build cell before this block.")

# ---------------------------------------------------------
# 1) Required survival columns
# ---------------------------------------------------------
req = ["Participant_ID", "time", "event"]
for c in req:
    if c not in df_model.columns:
        raise RuntimeError(f"Dataset is missing required column: {c}")

df_model = df_model.dropna(subset=["time", "event"]).copy()
df_model = df_model[df_model["time"] >= 0].copy()
df_model["event"] = df_model["event"].astype(int)

# ---------------------------------------------------------
# 2) Numeric features (exclude ID/time/event)
# ---------------------------------------------------------
drop_cols = {"Participant_ID", "time", "event"}
feat_cols = [
    c for c in df_model.columns
    if c not in drop_cols and pd.api.types.is_numeric_dtype(df_model[c])
]

# remove constants
feat_cols = [c for c in feat_cols if df_model[c].nunique(dropna=True) > 1]
if len(feat_cols) < 5:
    raise RuntimeError(f"Too few numeric features detected: {feat_cols}")

# ---------------------------------------------------------
# 3) Survival HOLDOUT split (stratified by event)
# ---------------------------------------------------------
train_idx, test_idx = train_test_split(
    df_model.index, test_size=0.25, random_state=42,
    stratify=df_model["event"].values
)
tr = df_model.loc[train_idx].copy()
te = df_model.loc[test_idx].copy()

# ---------------------------------------------------------
# 4) ANTI-LEAKAGE: remove post-baseline / follow-up / outcome-derived fields
# ---------------------------------------------------------
leak_patterns = [
    "days_to_death", "death", "deceased",
    "days_to_last_follow_up", "last_follow_up",
    "follow_up_time", "follow_ups", "follow_up", "days_to_follow_up",
    "time_to", "os_time", "pfs_time",
    "overall_survival", "progression_free"
]

feat_before = feat_cols[:]

# 4.1 drop by name patterns
feat_cols = [c for c in feat_cols if not any(p in c.lower() for p in leak_patterns)]
removed_by_pattern = sorted(list(set(feat_before) - set(feat_cols)))

# 4.2 near-zero variance in TRAIN
var_series = tr[feat_cols].var(numeric_only=True)
near_zero = var_series[var_series.fillna(0) < 1e-12].index.tolist()
feat_cols = [c for c in feat_cols if c not in near_zero]

# 4.3 highly correlated with time in TRAIN (often proxy/leakage)
tmp = tr[feat_cols + ["time"]].copy()
corr = tmp.corr(numeric_only=True)["time"].drop("time")
high_corr = corr[abs(corr) > 0.95].index.tolist()
feat_cols = [c for c in feat_cols if c not in high_corr]

# 4.4 too many missing values in TRAIN
na_rate = tr[feat_cols].isna().mean()
too_many_na = na_rate[na_rate > 0.60].index.tolist()
feat_cols = [c for c in feat_cols if c not in too_many_na]

print("\n[ANTI-LEAK SUMMARY]")
print("Features before:", len(feat_before))
print("Removed by pattern:", len(removed_by_pattern), "| ex:", removed_by_pattern[:8])
print("Removed near-zero var:", len(near_zero), "| ex:", near_zero[:8])
print("Removed corr(|time|)>0.95:", len(high_corr), "| ex:", high_corr[:8])
print("Removed NA>60%:", len(too_many_na), "| ex:", too_many_na[:8])
print("Features after anti-leak:", len(feat_cols))

if len(feat_cols) < 5:
    raise RuntimeError("Too few features after anti-leakage filters. Review patterns/filters or feature set.")

# keep a copy for ISS BEFORE Cox-only stabilization
feat_cols_iss = feat_cols[:]  # ISS uses the full anti-leak feature set

# save final anti-leak feature list (Methods / auditability)
feat_list_path = os.path.join(RES_DIR, "pred_features_used_survival.txt")
with open(feat_list_path, "w") as f:
    f.write("\n".join(feat_cols))
print("[OK] survival features saved:", feat_list_path)

# =========================================================
# 4B) EXTRA STABILITY FOR COX (SURVIVAL ONLY)
# - remove very rare binary features (low prevalence)
# - remove high collinearity among features
# - remove simple complete separation patterns (binary)
# =========================================================

# 1) remove very rare binary features in TRAIN (e.g., <3% ones)
rare_thresh = 0.03
rare = []
for c in feat_cols:
    s = tr[c]
    vals = set(pd.Series(s.dropna().unique()).tolist())
    if vals.issubset({0, 1}):
        freq = float(s.mean())
        if freq < rare_thresh:
            rare.append(c)

feat_cols = [c for c in feat_cols if c not in rare]
print(f"[COX-STAB] removed rare binaries (<{rare_thresh*100:.1f}% ones):", len(rare))

# 2) remove high collinearity among features (TRAIN)
Xcorr = tr[feat_cols].corr(numeric_only=True).abs()
upper = Xcorr.where(np.triu(np.ones(Xcorr.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]
feat_cols = [c for c in feat_cols if c not in to_drop]
print("[COX-STAB] removed by collinearity > 0.95:", len(to_drop))

# 3) remove binary features with (almost) complete separation wrt event
sep = []
for c in feat_cols:
    s = tr[c]
    vals = set(pd.Series(s.dropna().unique()).tolist())
    if vals.issubset({0, 1}):
        e1 = tr.loc[tr["event"] == 1, c].dropna()
        e0 = tr.loc[tr["event"] == 0, c].dropna()
        if len(e1) > 0 and len(e0) > 0:
            # if one group has almost no 1s or almost all 1s while the other is opposite
            p1 = float(e1.mean())
            p0 = float(e0.mean())
            if (p1 < 0.01 and p0 > 0.99) or (p0 < 0.01 and p1 > 0.99) or (p1 > 0.99 and p0 < 0.01) or (p0 > 0.99 and p1 < 0.01):
                sep.append(c)

feat_cols = [c for c in feat_cols if c not in sep]
print("[COX-STAB] removed by separation (binary):", len(sep), "| ex:", sep[:8])

print("[COX-STAB] survival features after stabilization:", len(feat_cols))

# ---------------------------------------------------------
# 5) Penalized Cox (CV for penalizer/l1_ratio) + TEST C-index
# ---------------------------------------------------------
def fit_eval_cox(train_df, test_df, feats, penalizer, l1_ratio):
    imp = SimpleImputer(strategy="median")
    sca = StandardScaler()

    Xtr = sca.fit_transform(imp.fit_transform(train_df[feats]))
    Xte = sca.transform(imp.transform(test_df[feats]))

    tr2 = train_df[["time", "event"]].copy()
    te2 = test_df[["time", "event"]].copy()
    tr2[feats] = Xtr
    te2[feats] = Xte

    try:
        cph = CoxPHFitter(penalizer=penalizer, l1_ratio=l1_ratio)
    except TypeError:
        cph = CoxPHFitter(penalizer=penalizer)

    cph.fit(tr2, duration_col="time", event_col="event")
    risk = cph.predict_partial_hazard(te2).values.ravel()
    cidx = concordance_index(te2["time"].values, -risk, te2["event"].values)
    return float(cidx), cph, risk

# avoid unpenalized Cox (often unstable)
pen_grid = [0.05, 0.1, 0.5, 1.0, 5.0]
l1_grid  = [0.0, 0.5]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_event = tr["event"].values

best = {"mean_cv_cindex": -np.inf, "penalizer": None, "l1_ratio": None}
rows = []

for pen in pen_grid:
    for l1 in l1_grid:
        scores = []
        for tr_i, va_i in cv.split(tr.index, y_event):
            df_tr = tr.iloc[tr_i]
            df_va = tr.iloc[va_i]
            try:
                cidx, _, _ = fit_eval_cox(df_tr, df_va, feat_cols, penalizer=pen, l1_ratio=l1)
                scores.append(cidx)
            except Exception:
                continue
        if scores:
            m = float(np.mean(scores))
            rows.append({"penalizer": pen, "l1_ratio": l1, "mean_cv_cindex": m, "n_folds_used": len(scores)})
            if m > best["mean_cv_cindex"]:
                best.update({"mean_cv_cindex": m, "penalizer": pen, "l1_ratio": l1})

cv_df = pd.DataFrame(rows).sort_values("mean_cv_cindex", ascending=False)
cv_path = os.path.join(RES_DIR, "pred_survival_cox_cv_grid.tsv")
cv_df.to_csv(cv_path, sep="\t", index=False)

cidx_test, cph_final, risk_test = fit_eval_cox(tr, te, feat_cols, best["penalizer"], best["l1_ratio"])

coef = cph_final.summary.reset_index().rename(columns={"index": "feature"})
coef_path = os.path.join(RES_DIR, "pred_survival_cox_coefficients.tsv")
coef.to_csv(coef_path, sep="\t", index=False)

risk_out = te[["Participant_ID", "time", "event"]].copy()
risk_out["risk_score"] = risk_test
risk_path = os.path.join(RES_DIR, "pred_survival_test_risk_scores.tsv")
risk_out.to_csv(risk_path, sep="\t", index=False)

perf_path = os.path.join(RES_DIR, "pred_survival_performance.tsv")
pd.DataFrame([{
    "model": "Penalized Cox (ANTI-LEAK)",
    "n_train": int(tr.shape[0]),
    "n_test": int(te.shape[0]),
    "events_train": int(tr["event"].sum()),
    "events_test": int(te["event"].sum()),
    "best_penalizer": best["penalizer"],
    "best_l1_ratio": best["l1_ratio"],
    "cv_mean_cindex": float(best["mean_cv_cindex"]),
    "test_cindex": float(cidx_test),
    "n_features": int(len(feat_cols))
}]).to_csv(perf_path, sep="\t", index=False)

print("\n[OK] Survival prediction (ANTI-LEAK)")
print(" - CV mean C-index:", round(best["mean_cv_cindex"], 4))
print(" - TEST C-index   :", round(cidx_test, 4))
print(" - cv:", cv_path)
print(" - coef:", coef_path)
print(" - risk:", risk_path)
print(" - perf:", perf_path)

# ---------------------------------------------------------
# 6) ISS prediction (if available) using FULL anti-leak feature set
# ---------------------------------------------------------
iss_candidates = ["diagnoses.iss_stage", "iss_stage", "ISS", "ISS_stage"]
iss_col = next((c for c in iss_candidates if c in df_model.columns), None)

if iss_col is None:
    print("\n[INFO] ISS stage not found. Skipping ISS classifier.")
else:
    df_iss = df_model.dropna(subset=[iss_col]).copy()
    y = df_iss[iss_col].astype(str).values
    X = df_iss[feat_cols_iss].copy()

    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=0.25, random_state=42, stratify=y
    )

    pipe = Pipeline(steps=[
        ("imp", SimpleImputer(strategy="median")),
        ("sca", StandardScaler()),
        ("sel", SelectKBest(score_func=mutual_info_classif, k=min(50, X_tr.shape[1]))),
        ("clf", LogisticRegression(
            max_iter=20000,
            solver="lbfgs",
            class_weight="balanced",
            multi_class="multinomial"
        ))
    ])

    # light tuning
    k_grid = [25, 50, 100]
    C_grid = [0.1, 1.0, 10.0]
    cv2 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    best2 = {"cv_macro_f1": -np.inf, "k": None, "C": None}
    for k in k_grid:
        for C in C_grid:
            pipe.set_params(sel__k=min(k, X_tr.shape[1]), clf__C=C)
            f1s = []
            for a, b in cv2.split(X_tr, y_tr):
                pipe.fit(X_tr.iloc[a], y_tr[a])
                pred = pipe.predict(X_tr.iloc[b])
                f1s.append(f1_score(y_tr[b], pred, average="macro"))
            m = float(np.mean(f1s))
            if m > best2["cv_macro_f1"]:
                best2.update({"cv_macro_f1": m, "k": k, "C": C})

    pipe.set_params(sel__k=min(best2["k"], X_tr.shape[1]), clf__C=best2["C"])
    pipe.fit(X_tr, y_tr)
    y_pred = pipe.predict(X_te)

    acc = accuracy_score(y_te, y_pred)
    f1m = f1_score(y_te, y_pred, average="macro")

    labels = np.unique(y)  # stable label ordering
    cm = confusion_matrix(y_te, y_pred, labels=labels)

    iss_perf_path = os.path.join(RES_DIR, "pred_iss_performance.tsv")
    pd.DataFrame([{
        "model": "LogReg + SelectKBest (ANTI-LEAK features)",
        "iss_col": iss_col,
        "best_k": best2["k"],
        "best_C": best2["C"],
        "cv_macro_f1": best2["cv_macro_f1"],
        "test_accuracy": float(acc),
        "test_macro_f1": float(f1m),
        "n_train": int(len(y_tr)),
        "n_test": int(len(y_te)),
        "n_features": int(X.shape[1])
    }]).to_csv(iss_perf_path, sep="\t", index=False)

    iss_rep_path = os.path.join(RES_DIR, "pred_iss_classification_report.txt")
    with open(iss_rep_path, "w") as f:
        f.write(classification_report(y_te, y_pred, zero_division=0))

    iss_cm_path = os.path.join(RES_DIR, "pred_iss_confusion_matrix.tsv")
    pd.DataFrame(cm, index=labels, columns=labels).to_csv(iss_cm_path, sep="\t")

    print("\n[OK] ISS prediction (ANTI-LEAK features)")
    print(" - TEST accuracy:", round(float(acc), 4), "| macro-F1:", round(float(f1m), 4))
    print(" - report:", iss_rep_path)
    print(" - cm:", iss_cm_path)
    print(" - perf:", iss_perf_path)


In [ ]:
# =========================================================
# ✅ FINAL) ZIPAR TUDO DO AMBIENTE DE TRABALHO + BAIXAR (COLAB)
# =========================================================
import os, zipfile, pathlib, datetime

# 1) Define "roots" do seu projeto (prioriza as pastas do pipeline)
roots = []
for k in ["BASE_OUT_DIR", "OUT_DIR", "RES_DIR", "LOG_DIR", "RAW_DIR", "PROC_DIR", "PROCESSED_DIR", "RESULTS_DIR"]:
    v = globals().get(k)
    if isinstance(v, str) and v and os.path.exists(v):
        roots.append(os.path.abspath(v))

# Fallback: se nada estiver definido, usa /content (Colab) ou cwd
if not roots:
    roots = ["/content"] if os.path.exists("/content") else [os.getcwd()]

# Remove duplicados mantendo ordem
seen = set()
roots = [r for r in roots if not (r in seen or seen.add(r))]

# 2) Nome do zip
ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
zip_path = os.path.join("/content" if os.path.exists("/content") else os.getcwd(),
                        f"workspace_backup_{ts}.zip")

# 3) O que excluir (para não explodir tamanho/tempo por cache/drive)
EXCLUDE_DIRS = {
    "/content/drive",              # evita zipar o Drive inteiro
    "/content/sample_data",
}
EXCLUDE_PARTS = {".ipynb_checkpoints", "__pycache__", ".cache", ".config", ".git"}

def should_skip(path: str) -> bool:
    ap = os.path.abspath(path)
    for ed in EXCLUDE_DIRS:
        if ap.startswith(os.path.abspath(ed) + os.sep) or ap == os.path.abspath(ed):
            return True
    parts = set(pathlib.Path(ap).parts)
    return len(parts.intersection(EXCLUDE_PARTS)) > 0

# 4) Zipar
count = 0
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED, compresslevel=6) as zf:
    for root in roots:
        root = os.path.abspath(root)
        if should_skip(root):
            continue
        for dirpath, dirnames, filenames in os.walk(root):
            # filtra diretórios excluídos
            dirnames[:] = [d for d in dirnames if not should_skip(os.path.join(dirpath, d))]
            if should_skip(dirpath):
                continue

            for fn in filenames:
                fp = os.path.join(dirpath, fn)
                if should_skip(fp):
                    continue

                # caminho relativo dentro do zip (mantém estrutura)
                arcname = os.path.relpath(fp, start=os.path.dirname(root))
                try:
                    zf.write(fp, arcname=arcname)
                    count += 1
                except Exception as e:
                    print(f"[WARN] não consegui adicionar: {fp} | {e}")

size_mb = os.path.getsize(zip_path) / (1024**2)
print(f"[OK] ZIP criado: {zip_path}")
print(f"[OK] Arquivos incluídos: {count}")
print(f"[OK] Tamanho: {size_mb:.2f} MB")

# 5) Baixar (Colab)
try:
    from google.colab import files
    files.download(zip_path)
except Exception as e:
    print("[INFO] Não consegui acionar download automático (fora do Colab?).")
    print("Faça download manual do arquivo:", zip_path)
    print("Detalhe:", str(e)[:200])
